In [1]:
import os, sys
import unidecode
import re
import json
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Ce notebook analyse les réponses aux questions en cherchant des liens entre les mots les plus utilisés, dans le but de comprendre les demandes les plus récurrentes des francais.

Dans un premier temps, les réponses sont préprocessées et transformées en liste de mots-clés. Puis, pour chaque réponse, úne analyse basique de la proximité des mots est réalisée. Bien que simple, cette methode permet de faire apparaitre des thèmes récurrents et de comprendre la teneur générale de ces demandes.

Les trois métriques utilisées sont :
+ co-occurence des mots : $co-occurence(i, j) = \sum_{users} 1_{i \in user} 1_{j \in user}$
+ confiance : $confiance(i -> j) = P(j | i)$
+ lift : $lift(i, j) = \frac{P(i \cap j)}{P(i)P(j)}$

La co-occurence permet d'évaluer les mots présents de manière commune, et donne une première idée des thèmes généraux.

La confiance donne une idée de la présence de $j$ quand $i$ est déjà présent : il surpondère les mots peu présents par rapport à la co-occurence.

Le lift estime le surplus de présence commune de $i$ et $j$ par rapport au cas où $i$ et $j$ sont indépendants. Cela permet de trouver des relations de causalité plus intéressantes, notamment dans des cas où certains mots sont peu présents mais très liés.

Pour chaque question :
+ On récupère les mots les plus employés sur ce thème
+ Pour chacun de ces mots, on regarde pour chaque métrique les mots les plus liés, ce qui donne une bonne idée des liens et des demandes

On peut ensuite raffiner l'analyse sur des mots moins présents mais porteurs de sens.

In [2]:
# function definitions

def compute_similarities(question_name, df_res):
    qu = df_res[question_name].dropna()
    df_qu_one_hot = compute_one_hot(qu)
    df_cov = compute_covariance(df_qu_one_hot)
    df_confidence = compute_confidence(df_cov)
    df_lift = compute_lift(df_cov, df_qu_one_hot)
    
    return qu, df_qu_one_hot, df_cov, df_confidence, df_lift
    
def compute_one_hot(qu):
    qu_words = []
    for i in qu:
        qu_words.extend(i)
    qu_words = list(set(qu_words))
    dic_qu_words = {word:i for i, word in enumerate(qu_words)}
    dic_qu_words_inverse = {i:word for i, word in enumerate(qu_words)}
    
    qu_one_hot = np.zeros((len(qu), len(qu_words)), dtype=bool)
    for i, li_word in enumerate(qu):
        for word in li_word:
            qu_one_hot[i, dic_qu_words[word]] = True
            
    df_qu_one_hot = pd.DataFrame(columns=qu_words, data=qu_one_hot).to_sparse(fill_value=False)

    
    temp_mask = np.sum(df_qu_one_hot.values, axis=0, dtype=np.uint16) > 250
    words_enough = df_qu_one_hot.columns.values[temp_mask]
    df_qu_one_hot = df_qu_one_hot[words_enough].to_sparse(fill_value=0)
    
    del temp_mask
    del qu_one_hot
    gc.collect()
    
    return df_qu_one_hot

def compute_covariance(df_qu_one_hot):
    N = df_qu_one_hot.shape[1]
    
    cov = np.zeros((N, N), dtype=np.uint16)
    np.matmul(df_qu_one_hot.values.T, df_qu_one_hot.values, out=cov)
    
    df_cov = pd.DataFrame(columns=df_qu_one_hot.columns, index=df_qu_one_hot.columns, data=cov) * (cov > 100)
    del cov
    gc.collect()
    
    return df_cov

def compute_confidence(df_cov):
    df_confidence = np.diag(1 / np.diag(df_cov)).dot(df_cov).astype(np.float32)
    df_confidence = pd.DataFrame(index=df_cov.index, columns=df_cov.index, data=df_confidence)
    return df_confidence

def compute_lift(df_cov, df_qu_one_hot):
    inv_diag_cov = 1 / np.diag(df_cov.values)
    lift = inv_diag_cov.reshape((-1, 1)) * df_cov.values * inv_diag_cov.reshape((1, -1))
    df_lift = pd.DataFrame(columns=df_qu_one_hot.columns,
                             index=df_qu_one_hot.columns,
                             data=lift).replace(0.0, np.nan).astype(np.float32)
    del lift
    del inv_diag_cov
    gc.collect()
    
    return df_lift

# Loading of files and preprocessing

In [3]:
PATH_TO_FILE = "./proposals_cleaned/services.json"

In [4]:
with open(PATH_TO_FILE, "r") as f:
    raw_json = [s.strip() for s in f.readlines()]
    
parsed_json = [json.loads(raw_json[i]) for i in range(len(raw_json))]
del raw_json

In [5]:
col = ["titre"]
for ques in parsed_json[0]["responses"]:
    col.append(ques["question"]["id"]) 
        
data = []
for elt in parsed_json:
    resp = {"titre" : elt["titre"]}
    for ques in elt["responses"]:
        if ques["question"]["title"] == "Si oui, comment ?":
            if ques["question"]["id"] == "185":
                ques["question"]["title"] += "_terrain"
            else:
                ques["question"]["title"] += "_admin"
        try:
            text = json.loads((ques["value"]))
            resp[ques["question"]["title"]] = ''
            if text["labels"] != []:
                resp[ques["question"]["title"]] += ", ".join(text["labels"])
            if text["other"] is not None:
                resp[ques["question"]["title"]] += ", " + text["other"]
            if resp[ques["question"]["title"]] == '':
                resp[ques["question"]["title"]] = np.nan
        except:
            if ques["value"] is not None and ques["value"][0] != "{":
                resp[ques["question"]["title"]] = ques["value"]
            else:
                resp[ques["question"]["title"]] = np.nan
    data.append(resp)

In [6]:
res = pd.DataFrame.from_dict(data)
del data
gc.collect()

0

In [7]:
res.notna().mean(axis=0).sort_values(ascending=False)

titre                                                                                                                                                                                                                                 1.000000
Estimez-vous avoir accès aux services publics dont vous avez besoin ?                                                                                                                                                                 0.720851
Que pensez-vous de l'organisation de l'Etat et des administrations en France ? De quelle manière cette organisation devrait-elle évoluer ?                                                                                            0.712880
Connaissez-vous le "droit à l'erreur", c'est-à-dire le droit d'affirmer votre bonne foi lorsque vous faites un erreur dans vos déclarations ?                                                                                         0.696867
Avez-vous déjà utilisé certaines de ces nouv

# Find important words

In [8]:
li_word_count = []
for elt in res.values:
    for item in elt:
        if type(item) == str:
            li_word_count.extend([unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item)])

In [9]:
dic_word_count = {}
for i in li_word_count:
    if len(i) > 0:
        if i in dic_word_count.keys():
            dic_word_count[i] +=1
        else:
            dic_word_count[i] = 1

In [10]:
df_word_count = pd.DataFrame.from_dict(dic_word_count, orient="index")
df_word_count.sort_values(0, ascending=False)
df_word_count.columns = ["freq_in_debate"]

In [11]:
del li_word_count
del dic_word_count

gc.collect()

202

In [12]:
df_word_count = df_word_count[df_word_count["freq_in_debate"] > 100]

In [13]:
li_stopwords = [unidecode.unidecode(x.strip().lower()) for x in open("french_stopwords.txt", 'r').readlines()]
li_stopwords = set(li_stopwords)
li_stopwords.remove("aucun")
li_stopwords.remove("tout")
li_stopwords.remove("tous")
li_stopwords.remove("non")

In [14]:
new_data = []
for i, elt in enumerate(res.index):
    temp_res = []
    for col in res.columns:
        item = res.loc[elt, col]
        if type(item) == str:
            li_item = [unidecode.unidecode(x.lower()) for x in re.findall(r'\w+', item) if unidecode.unidecode(x.lower()) not in li_stopwords]
            temp_res.append(li_item)
        else:
            temp_res.append(None)
    new_data.append(temp_res)

In [15]:
new_res = pd.DataFrame(new_data, columns=res.columns).to_sparse()
del new_data
gc.collect()

0

# Question 1 :   Que pensez-vous de l'organisation de l'Etat et des administrations en France ? De quelle manière cette organisation devrait-elle évoluer ?

In [16]:
question_1 = "Que pensez-vous de l'organisation de l'Etat et des administrations en France ? De quelle manière cette organisation devrait-elle évoluer ?"
qu_1,df_qu_1_one_hot,df_cov_1,df_confidence_1,df_lift_1 = compute_similarities(question_1,new_res)

In [17]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_1[i].sort_values(ascending=False).head(10))

faut


faut         5247
faire        1471
tout         1428
citoyens     1036
services      997
supprimer     973
service       968
temps         862
communes      853
nombre        843
Name: faut, dtype: uint16

services


services           4100
publics            1310
faut                997
faire               806
tous                791
tout                760
service             726
administrations     676
fonctionnaires      671
communes            662
Name: services, dtype: uint16

faire


...

france          2147
faut             651
faire            620
tout             581
tous             521
nombre           484
pays             473
organisation     471
services         451
regions          450
Name: france, dtype: uint16

non


non               2118
faire              707
tout               610
faut               605
tous               575
services           514
fonctionnaires     476
nombre             470
citoyens           467
elus               422
Name: non, dtype: uint16

region


region          2068
departement     1098
communes         855
commune          744
regions          546
faut             530
supprimer        496
departements     483
nombre           421
services         414
Name: region, dtype: uint16

In [18]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_1[i].sort_values(ascending=False).head(10))

faut


faut           1.000000
probants       0.997608
flashent       0.995169
rallonges      0.992840
biaisee        0.990499
colles         0.988152
rives          0.987952
abaissement    0.983871
injustement    0.981265
detourner      0.976359
Name: faut, dtype: float32

services


services      1.000000
publics       0.792019
publiques     0.497613
hopitaux      0.483412
acces         0.469242
qualite       0.455157
sociaux       0.452107
transports    0.431611
zones         0.423676
proximite     0.405900
Name: services, dtype: float32

faire


...

france       1.000000
allemagne    0.448052
millions     0.440945
europe       0.405467
000          0.376168
enfants      0.345890
6            0.343465
pays         0.336895
grands       0.333333
10           0.320166
Name: france, dtype: float32

non


non        1.000000
6          0.410334
cumul      0.388112
enfants    0.376712
mandats    0.364902
mandat     0.356459
4          0.355899
30         0.354949
vote       0.352645
droit      0.347899
Name: non, dtype: float32

region


region              1.000000
departement         0.568617
metropole           0.561338
communaute          0.506747
commune             0.481553
intercommunalite    0.466934
departemental       0.318627
nouvelle            0.289694
collectivite        0.289398
maire               0.286064
Name: region, dtype: float32

In [19]:
for i in df_qu_1_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_1[i].sort_values(ascending=False).head(10))

faut


faut           0.000191
probants       0.000190
flashent       0.000190
rallonges      0.000189
biaisee        0.000189
colles         0.000188
rives          0.000188
abaissement    0.000188
injustement    0.000187
detourner      0.000186
Name: faut, dtype: float32

services


services      0.000244
publics       0.000193
publiques     0.000121
hopitaux      0.000118
acces         0.000114
qualite       0.000111
sociaux       0.000110
transports    0.000105
zones         0.000103
proximite     0.000099
Name: services, dtype: float32

faire


...

france       0.000466
allemagne    0.000209
millions     0.000205
europe       0.000189
000          0.000175
enfants      0.000161
6            0.000160
pays         0.000157
grands       0.000155
10           0.000149
Name: france, dtype: float32

non


non        0.000472
6          0.000194
cumul      0.000183
enfants    0.000178
mandats    0.000172
mandat     0.000168
4          0.000168
30         0.000168
vote       0.000166
droit      0.000164
Name: non, dtype: float32

region


region              0.000484
departement         0.000275
metropole           0.000271
communaute          0.000245
commune             0.000233
intercommunalite    0.000226
departemental       0.000154
nouvelle            0.000140
collectivite        0.000140
maire               0.000138
Name: region, dtype: float32

In [20]:
gc.collect()

147

# Question 2 :  Selon vous, l'Etat doit-il aujourd'hui transférer de nouvelles missions aux collectivités territoriales ?

In [21]:
question_2 = "Selon vous, l'Etat doit-il aujourd'hui transférer de nouvelles missions aux collectivités territoriales ?"
qu_2,df_qu_2_one_hot,df_cov_2,df_confidence_2,df_lift_2 = compute_similarities(question_2,new_res)

In [22]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_2[i].sort_values(ascending=False).head(10))

oui


oui    14901
non        0
Name: oui, dtype: uint16

non


non    14111
oui        0
Name: non, dtype: uint16

In [23]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_2[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [24]:
for i in df_qu_2_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_2[i].sort_values(ascending=False).head(10))

oui


oui    0.000067
non         NaN
Name: oui, dtype: float32

non


non    0.000071
oui         NaN
Name: non, dtype: float32

In [25]:
gc.collect()

14

# Question 3 : Si oui, lesquelles ?

In [26]:
question_3 = "Si oui, lesquelles ?"
qu_3,df_qu_3_one_hot,df_cov_3,df_confidence_3,df_lift_3 = compute_similarities(question_3, new_res)

In [27]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_3[i].sort_values(ascending=False).head(10))

tout


tout             1879
regalien          319
concerne          245
collectivites     236
regions           226
faut              201
education         188
vie               179
territoire        177
faire             170
Name: tout, dtype: uint16

collectivites


collectivites    1695
territoriales     815
missions          422
locales           334
faut              263
tout              236
moyens            223
non               185
transferer        180
faire             176
Name: collectivites, dtype: uint16

missions


...

niveau           802
local            180
regions          175
tout             162
national         142
collectivites    127
faut             127
missions         126
gestion          111
education        109
Name: niveau, dtype: uint16

faire


faire            782
collectivites    176
tout             170
faut             169
regions          115
citoyens         104
france             0
emploi             0
cas                0
decisions          0
Name: faire, dtype: uint16

securite


securite     775
routiere     301
education    193
sante        164
tout         153
justice      125
france         0
proximite      0
routes         0
gestion        0
Name: securite, dtype: uint16

In [28]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_3[i].sort_values(ascending=False).head(10))

tout


tout           1.000000
concerne       0.712209
regalien       0.635458
vie            0.389978
justice        0.309582
local          0.264591
citoyens       0.239631
faire          0.217391
territoires    0.215074
faut           0.209812
Name: tout, dtype: float32

collectivites


collectivites    1.000000
territoriales    0.903548
locales          0.505295
nouvelles        0.434932
transferer       0.330882
transfert        0.319202
donner           0.293307
competences      0.278992
missions         0.278916
faut             0.274530
Name: collectivites, dtype: float32

missions


...

niveau       1.000000
national     0.471761
local        0.350195
faut         0.132568
region       0.132282
exemple      0.123821
regions      0.118483
gestion      0.090984
tout         0.086216
education    0.084825
Name: niveau, dtype: float32

faire


faire            1.000000
faut             0.176409
citoyens         0.159754
collectivites    0.103835
tout             0.090474
regions          0.077861
france           0.000000
emploi           0.000000
cas              0.000000
decisions        0.000000
Name: faire, dtype: float32

securite


securite     1.000000
routiere     0.756281
justice      0.307125
sante        0.185730
education    0.150195
tout         0.081426
france       0.000000
proximite    0.000000
routes       0.000000
gestion      0.000000
Name: securite, dtype: float32

In [29]:
for i in df_qu_3_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_3[i].sort_values(ascending=False).head(10))

tout


tout           0.000532
concerne       0.000379
regalien       0.000338
vie            0.000208
justice        0.000165
local          0.000141
citoyens       0.000128
faire          0.000116
territoires    0.000114
faut           0.000112
Name: tout, dtype: float32

collectivites


collectivites    0.000590
territoriales    0.000533
locales          0.000298
nouvelles        0.000257
transferer       0.000195
transfert        0.000188
donner           0.000173
competences      0.000165
missions         0.000165
faut             0.000162
Name: collectivites, dtype: float32

missions


...

niveau       0.001247
national     0.000588
local        0.000437
faut         0.000165
region       0.000165
exemple      0.000154
regions      0.000148
gestion      0.000113
tout         0.000108
education    0.000106
Name: niveau, dtype: float32

faire


faire            0.001279
faut             0.000226
citoyens         0.000204
collectivites    0.000133
tout             0.000116
regions          0.000100
non                   NaN
decision              NaN
securite              NaN
sais                  NaN
Name: faire, dtype: float32

securite


securite     0.001290
routiere     0.000976
justice      0.000396
sante        0.000240
education    0.000194
tout         0.000105
non               NaN
decision          NaN
sais              NaN
locales           NaN
Name: securite, dtype: float32

In [30]:
gc.collect()

147

# Question 4 :  Estimez-vous avoir accès aux services publics dont vous avez besoin ?

In [31]:
question_4 = "Estimez-vous avoir accès aux services publics dont vous avez besoin ?"
qu_4,df_qu_4_one_hot,df_cov_4,df_confidence_4,df_lift_4 = compute_similarities(question_4, new_res)

In [32]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_4[i].sort_values(ascending=False).head(10))

oui


oui    21197
non        0
Name: oui, dtype: uint16

non


non    10000
oui        0
Name: non, dtype: uint16

In [33]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_4[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [34]:
for i in df_qu_4_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_4[i].sort_values(ascending=False).head(10))

oui


oui    0.000047
non         NaN
Name: oui, dtype: float32

non


non    0.0001
oui       NaN
Name: non, dtype: float32

In [35]:
gc.collect()

14

# Question 5 : Si non, quels types de services publics vous manquent dans votre territoire et qu'il est nécessaire de renforcer ?

In [36]:
question_5 = "Si non, quels types de services publics vous manquent dans votre territoire et qu'il est nécessaire de renforcer ?"
qu_5,df_qu_5_one_hot,df_cov_5,df_confidence_5,df_lift_5 = compute_similarities(question_5, new_res)

In [37]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_5[i].sort_values(ascending=False).head(10))

services


services    2891
publics     1197
acces        492
tous         449
sante        401
poste        386
faut         369
service      346
public       334
internet     323
Name: services, dtype: uint16

poste


poste         1466
services       386
impots         236
service        206
public         180
transports     169
publics        166
hopitaux       133
horaires       125
sante          122
Name: poste, dtype: uint16

service


...

manque      644
services    208
service     135
medecins    124
publics     109
sante       101
ecoles        0
cas           0
identite      0
grise         0
Name: manque, dtype: uint16

proximite


proximite    611
services     257
publics      120
service      110
poste        108
hopitaux     102
sante        101
cas            0
identite       0
grise          0
Name: proximite, dtype: uint16

commun


commun        592
transports    422
transport     183
services      107
soins           0
communes        0
cas             0
identite        0
grise           0
emploi          0
Name: commun, dtype: uint16

In [38]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_5[i].sort_values(ascending=False).head(10))

services


services     1.000000
publics      0.842365
publiques    0.677852
citoyens     0.513793
maisons      0.496933
tous         0.475132
communes     0.456026
besoin       0.433071
villes       0.423611
proximite    0.420622
Name: services, dtype: float32

poste


poste         1.000000
impots        0.278960
mairie        0.277333
horaires      0.228519
hopital       0.207739
hopitaux      0.180461
proximite     0.176759
public        0.175097
transports    0.147341
service       0.140710
Name: poste, dtype: float32

service


...

manque      1.000000
medecins    0.186186
service     0.092213
sante       0.080350
publics     0.076707
services    0.071947
ecoles      0.000000
cas         0.000000
identite    0.000000
grise       0.000000
Name: manque, dtype: float32

proximite


proximite    1.000000
hopitaux     0.138399
services     0.088897
publics      0.084448
sante        0.080350
service      0.075137
poste        0.073670
cas          0.000000
identite     0.000000
grise        0.000000
Name: proximite, dtype: float32

commun


commun        1.000000
transports    0.367916
transport     0.326786
services      0.037011
soins         0.000000
communes      0.000000
cas           0.000000
identite      0.000000
grise         0.000000
emploi        0.000000
Name: commun, dtype: float32

In [39]:
for i in df_qu_5_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_5[i].sort_values(ascending=False).head(10))

services


services     0.000346
publics      0.000291
publiques    0.000234
citoyens     0.000178
maisons      0.000172
tous         0.000164
communes     0.000158
besoin       0.000150
villes       0.000147
proximite    0.000145
Name: services, dtype: float32

poste


poste         0.000682
impots        0.000190
mairie        0.000189
horaires      0.000156
hopital       0.000142
hopitaux      0.000123
proximite     0.000121
public        0.000119
transports    0.000101
service       0.000096
Name: poste, dtype: float32

service


...

manque      0.001553
medecins    0.000289
service     0.000143
sante       0.000125
publics     0.000119
services    0.000112
non              NaN
caf              NaN
securite         NaN
habite           NaN
Name: manque, dtype: float32

proximite


proximite    0.001637
hopitaux     0.000227
services     0.000145
publics      0.000138
sante        0.000132
service      0.000123
poste        0.000121
non               NaN
caf               NaN
securite          NaN
Name: proximite, dtype: float32

commun


commun        0.001689
transports    0.000621
transport     0.000552
services      0.000063
non                NaN
caf                NaN
securite           NaN
habite             NaN
telephone          NaN
carte              NaN
Name: commun, dtype: float32

In [40]:
gc.collect()

285

# Question 6 : Quels nouveaux services ou quelles démarches souhaitez-vous voir développées sur Internet en priorité ?

In [41]:
question_6 = "Quels nouveaux services ou quelles démarches souhaitez-vous voir développées sur Internet en priorité ?"
qu_6,df_qu_6_one_hot,df_cov_6,df_confidence_6,df_lift_6 = compute_similarities(question_6, new_res)

In [42]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_6[i].sort_values(ascending=False).head(10))

internet


internet     4931
tout         1199
services     1148
demarches    1065
acces         871
faut          746
tous          714
faire         638
pouvoir       505
service       438
Name: internet, dtype: uint16

services


services     2667
internet     1148
tous          751
publics       667
acces         474
demarches     423
faut          405
tout          377
service       270
faire         244
Name: services, dtype: uint16

demarches


...

impots       640
internet     195
services     155
demarches    131
tout         126
caf          104
publics        0
demarche       0
vote           0
proximite      0
Name: impots, dtype: uint16

numerique


numerique         603
tout              177
internet          176
services          142
faut              115
demarches         113
acces             103
identite            0
administratifs      0
accessible          0
Name: numerique, dtype: uint16

public


public            593
service           391
internet          237
services          183
tous              103
tout              103
faut              101
caf                 0
grise               0
administratifs      0
Name: public, dtype: uint16

In [43]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_6[i].sort_values(ascending=False).head(10))

internet


internet       1.000000
monde          0.775362
utiliser       0.737828
agees          0.681128
devraient      0.671642
accessibles    0.654237
accessible     0.626712
developper     0.610548
faudrait       0.603217
faut           0.563870
Name: internet, dtype: float32

services


services          1.000000
publics           0.857327
accessibles       0.511864
proximite         0.490842
developper        0.442191
tous              0.392782
administratifs    0.359322
faudrait          0.353887
devraient         0.350746
public            0.308600
Name: services, dtype: float32

demarches


...

impots       1.000000
caf          0.351351
services     0.058118
demarches    0.054561
tout         0.053458
internet     0.039546
publics      0.000000
vote         0.000000
demarche     0.000000
proximite    0.000000
Name: impots, dtype: float32

numerique


numerique         1.000000
faut              0.086924
tout              0.075095
acces             0.066153
services          0.053243
demarches         0.047064
internet          0.035693
identite          0.000000
administratifs    0.000000
accessible        0.000000
Name: numerique, dtype: float32

public


public        1.000000
service       0.325562
faut          0.076342
services      0.068616
tous          0.053870
internet      0.048063
tout          0.043700
securite      0.000000
developper    0.000000
demarche      0.000000
Name: public, dtype: float32

In [44]:
for i in df_qu_6_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_6[i].sort_values(ascending=False).head(10))

internet


internet       0.000203
monde          0.000157
utiliser       0.000150
agees          0.000138
devraient      0.000136
accessibles    0.000133
accessible     0.000127
developper     0.000124
faudrait       0.000122
faut           0.000114
Name: internet, dtype: float32

services


services          0.000375
publics           0.000321
accessibles       0.000192
proximite         0.000184
developper        0.000166
tous              0.000147
administratifs    0.000135
faudrait          0.000133
devraient         0.000132
public            0.000116
Name: services, dtype: float32

demarches


...

impots        0.001563
caf           0.000549
services      0.000091
demarches     0.000085
tout          0.000084
internet      0.000062
non                NaN
numerique          NaN
securite           NaN
developper         NaN
Name: impots, dtype: float32

numerique


numerique    0.001658
faut         0.000144
tout         0.000125
acces        0.000110
services     0.000088
demarches    0.000078
internet     0.000059
non               NaN
caf               NaN
securite          NaN
Name: numerique, dtype: float32

public


public       0.001686
service      0.000549
faut         0.000129
services     0.000116
tous         0.000091
internet     0.000081
tout         0.000074
non               NaN
numerique         NaN
caf               NaN
Name: public, dtype: float32

In [45]:
gc.collect()

147

# Question 7 :  Avez-vous déjà utilisé certaines de ces nouvelles formes de services publics ?

In [46]:
question_7 = "Avez-vous déjà utilisé certaines de ces nouvelles formes de services publics ?"
qu_7,df_qu_7_one_hot,df_cov_7,df_confidence_7,df_lift_7 = compute_similarities(question_7, new_res)

In [47]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_7[i].sort_values(ascending=False).head(10))

non


non    21522
oui        0
Name: non, dtype: uint16

oui


oui    8334
non       0
Name: oui, dtype: uint16

In [48]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_7[i].sort_values(ascending=False).head(10))

non


non    1.0
oui    0.0
Name: non, dtype: float32

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

In [49]:
for i in df_qu_7_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_7[i].sort_values(ascending=False).head(10))

non


non    0.000046
oui         NaN
Name: non, dtype: float32

oui


oui    0.00012
non        NaN
Name: oui, dtype: float32

In [50]:
gc.collect()

14

# Question 8 :  Si oui, en avez-vous été satisfait ?

In [51]:
question_8 = "Si oui, en avez-vous été satisfait ?"
qu_8,df_qu_8_one_hot,df_cov_8,df_confidence_8,df_lift_8 = compute_similarities(question_8, new_res)

In [52]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_8[i].sort_values(ascending=False).head(10))

oui


oui    6185
non       0
Name: oui, dtype: uint16

non


non    2994
oui       0
Name: non, dtype: uint16

In [53]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_8[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [54]:
for i in df_qu_8_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_8[i].sort_values(ascending=False).head(10))

oui


oui    0.000162
non         NaN
Name: oui, dtype: float32

non


non    0.000334
oui         NaN
Name: non, dtype: float32

In [55]:
gc.collect()

14

# Question 9 : Quelles améliorations préconiseriez-vous ?

In [56]:
question_9 = "Quelles améliorations préconiseriez-vous ?"
qu_9,df_qu_9_one_hot,df_cov_9,df_confidence_9,df_lift_9 = compute_similarities(question_9, new_res)

In [57]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_9[i].sort_values(ascending=False).head(10))

services


services      2579
publics       1125
public         410
maisons        375
service        316
tous           285
agents         268
faut           255
developper     249
faire          223
Name: services, dtype: uint16

publics


publics        1352
services       1125
agents          345
maisons         229
service         201
polyvalents     194
public          186
tous            159
faut            143
formes          143
Name: publics, dtype: uint16

service


...

prise        377
rendez       208
service      106
public         0
formation      0
bus            0
tout           0
travail        0
publics        0
communes       0
Name: prise, dtype: uint16

formes


formes       373
services     169
publics      143
agents         0
personnel      0
formation      0
bus            0
tout           0
travail        0
communes       0
Name: formes, dtype: uint16

fonctionnaires


fonctionnaires    362
public              0
formation           0
bus                 0
tout                0
travail             0
publics             0
communes            0
proximite           0
agents              0
Name: fonctionnaires, dtype: uint16

In [58]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_9[i].sort_values(ascending=False).head(10))

services


services      1.000000
publics       0.832101
maisons       0.684307
bus           0.596958
lieu          0.556364
tous          0.489691
proximite     0.472313
developper    0.456881
formes        0.453083
place         0.411371
Name: services, dtype: float32

publics


publics        1.000000
polyvalents    0.555874
bus            0.460076
services       0.436216
maisons        0.417883
formes         0.383378
agents         0.342602
tous           0.273196
acces          0.245977
developper     0.238532
Name: publics, dtype: float32

service


...

prise        1.000000
rendez       0.325509
service      0.082555
public       0.000000
formation    0.000000
bus          0.000000
tout         0.000000
travail      0.000000
publics      0.000000
communes     0.000000
Name: prise, dtype: float32

formes


formes       1.000000
publics      0.105769
services     0.065529
agents       0.000000
personnel    0.000000
formation    0.000000
bus          0.000000
tout         0.000000
travail      0.000000
communes     0.000000
Name: formes, dtype: float32

fonctionnaires


fonctionnaires    1.0
public            0.0
formation         0.0
bus               0.0
tout              0.0
travail           0.0
publics           0.0
communes          0.0
proximite         0.0
agents            0.0
Name: fonctionnaires, dtype: float32

In [59]:
for i in df_qu_9_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_9[i].sort_values(ascending=False).head(10))

services


services      0.000388
publics       0.000323
maisons       0.000265
bus           0.000231
lieu          0.000216
tous          0.000190
proximite     0.000183
developper    0.000177
formes        0.000176
place         0.000160
Name: services, dtype: float32

publics


publics        0.000740
polyvalents    0.000411
bus            0.000340
services       0.000323
maisons        0.000309
formes         0.000284
agents         0.000253
tous           0.000202
acces          0.000182
developper     0.000176
Name: publics, dtype: float32

service


...

prise          0.002653
rendez         0.000863
service        0.000219
non                 NaN
developper          NaN
horaires            NaN
impots              NaN
pouvoir             NaN
demarches           NaN
polyvalents         NaN
Name: prise, dtype: float32

formes


formes        0.002681
publics       0.000284
services      0.000176
non                NaN
developper         NaN
horaires           NaN
impots             NaN
pouvoir            NaN
demarches          NaN
service            NaN
Name: formes, dtype: float32

fonctionnaires


fonctionnaires    0.002762
non                    NaN
developper             NaN
horaires               NaN
impots                 NaN
pouvoir                NaN
demarches              NaN
service                NaN
polyvalents            NaN
formes                 NaN
Name: fonctionnaires, dtype: float32

In [60]:
gc.collect()

294

# Question 10 : Quand vous pensez à l'évolution des services publics au cours des dernières années, quels sont ceux qui ont évolué de manière positive ?

In [61]:
question_10 = "Quand vous pensez à l'évolution des services publics au cours des dernières années, quels sont ceux qui ont évolué de manière positive ?"
qu_10,df_qu_10_one_hot,df_cov_10,df_confidence_10,df_lift_10 = compute_similarities(question_10, new_res)

In [62]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_10[i].sort_values(ascending=False).head(10))

impots


impots         5614
internet        775
service         751
services        637
ligne           514
declaration     470
securite        410
sociale         402
poste           396
site            267
Name: impots, dtype: uint16

services


services    3279
publics      989
impots       637
internet     536
fiscaux      448
tous         340
service      278
ligne        263
poste        255
public       236
Name: services, dtype: uint16

aucun


...

evolution          692
positive           223
services           212
publics            118
internet           116
emploi               0
agents               0
identite             0
grise                0
administratives      0
Name: evolution, dtype: uint16

faire


faire          670
internet       173
services       165
demarches      146
impots         117
tout           107
service        102
publics          0
declaration      0
cpam             0
Name: faire, dtype: uint16

acces


acces          622
internet       330
services       190
impots         124
publics          0
declaration      0
cpam             0
ligne            0
pole             0
aucun            0
Name: acces, dtype: uint16

In [63]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_10[i].sort_values(ascending=False).head(10))

impots


impots         1.000000
gouv           0.678700
site           0.588106
declaration    0.586035
source         0.515254
sociale        0.459954
securite       0.436635
caf            0.430556
service        0.427433
ligne          0.424092
Name: impots, dtype: float32

services


services     1.000000
fiscaux      0.919918
publics      0.914048
publiques    0.394022
maniere      0.359133
tous         0.349076
evolue       0.346154
agents       0.335526
exemple      0.332609
positive     0.330629
Name: services, dtype: float32

aucun


...

evolution          1.000000
positive           0.452333
publics            0.109057
services           0.064654
internet           0.044326
emploi             0.000000
agents             0.000000
identite           0.000000
grise              0.000000
administratives    0.000000
Name: evolution, dtype: float32

faire


faire          1.000000
tout           0.125000
demarches      0.118506
internet       0.066106
service        0.058054
services       0.050320
impots         0.020841
ligne          0.000000
pole           0.000000
degradation    0.000000
Name: faire, dtype: float32

acces


acces          1.000000
internet       0.126099
services       0.057944
impots         0.022088
publics        0.000000
declaration    0.000000
cpam           0.000000
ligne          0.000000
pole           0.000000
aucun          0.000000
Name: acces, dtype: float32

In [64]:
for i in df_qu_10_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_10[i].sort_values(ascending=False).head(10))

impots


impots         0.000178
gouv           0.000121
site           0.000105
declaration    0.000104
source         0.000092
sociale        0.000082
securite       0.000078
caf            0.000077
service        0.000076
ligne          0.000076
Name: impots, dtype: float32

services


services     0.000305
fiscaux      0.000281
publics      0.000279
publiques    0.000120
maniere      0.000110
tous         0.000106
evolue       0.000106
agents       0.000102
exemple      0.000101
positive     0.000101
Name: services, dtype: float32

aucun


...

evolution    0.001445
positive     0.000654
publics      0.000158
services     0.000093
internet     0.000064
non               NaN
numerique         NaN
caf               NaN
source            NaN
securite          NaN
Name: evolution, dtype: float32

faire


faire        0.001493
tout         0.000187
demarches    0.000177
internet     0.000099
service      0.000087
services     0.000075
impots       0.000031
non               NaN
numerique         NaN
caf               NaN
Name: faire, dtype: float32

acces


acces        0.001608
internet     0.000203
services     0.000093
impots       0.000036
non               NaN
numerique         NaN
caf               NaN
source            NaN
evolution         NaN
securite          NaN
Name: acces, dtype: float32

In [65]:
gc.collect()

147

# Question 11 : Quels sont les services publics qui doivent le plus évoluer selon vous ?

In [66]:
question_11 = "Quels sont les services publics qui doivent le plus évoluer selon vous ?"
qu_11,df_qu_11_one_hot,df_cov_11,df_confidence_11,df_lift_11 = compute_similarities(question_11, new_res)

In [67]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_11[i].sort_values(ascending=False).head(10))

services


services     3410
publics       746
tous          642
sante         567
service       357
education     316
hopitaux      292
faut          283
public        269
faire         252
Name: services, dtype: uint16

tous


tous         3198
services      642
service       280
faut          262
publics       251
education     243
faire         218
sante         211
public        206
tout          197
Name: tous, dtype: uint16

education


...

faut         1066
services      283
tous          262
faire         226
education     204
service       186
tout          186
sante         177
public        162
publics       148
Name: faut, dtype: uint16

tout


tout         1046
services      249
tous          197
faut          186
faire         183
education     182
service       163
sante         162
monde         129
public        128
Name: tout, dtype: uint16

police


police         1027
justice         353
education       237
gendarmerie     202
services        186
sante           185
nationale       176
hopitaux        143
voir              0
communes          0
Name: police, dtype: uint16

In [68]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_11[i].sort_values(ascending=False).head(10))

services


services          1.000000
publics           0.739346
sociaux           0.728302
administratifs    0.549618
proximite         0.433010
publiques         0.425856
evoluer           0.359914
exemple           0.335249
medecins          0.301408
acces             0.294355
Name: services, dtype: float32

tous


tous        1.000000
evoluer     0.308190
citoyens    0.306931
non         0.278912
publics     0.248761
exemple     0.247126
faut        0.245779
internet    0.244493
faire       0.223819
travail     0.205179
Name: tous, dtype: float32

education


...

faut        1.000000
exemple     0.239464
faire       0.232033
moyens      0.200000
non         0.190476
tout        0.177820
public      0.148760
publics     0.146680
service     0.115099
services    0.082991
Name: faut, dtype: float32

tout


tout         1.000000
monde        0.486792
temps        0.243736
non          0.207483
faire        0.187885
faut         0.174484
public       0.117539
service      0.100866
services     0.073021
nationale    0.061923
Name: tout, dtype: float32

police


police         1.000000
gendarmerie    0.798419
justice        0.196439
hopitaux       0.095909
nationale      0.095600
education      0.075048
sante          0.063205
services       0.054545
cartes         0.000000
charge         0.000000
Name: police, dtype: float32

In [69]:
for i in df_qu_11_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_11[i].sort_values(ascending=False).head(10))

services


services          0.000293
publics           0.000217
sociaux           0.000214
administratifs    0.000161
proximite         0.000127
publiques         0.000125
evoluer           0.000106
exemple           0.000098
medecins          0.000088
acces             0.000086
Name: services, dtype: float32

tous


tous        0.000313
evoluer     0.000096
citoyens    0.000096
non         0.000087
publics     0.000078
exemple     0.000077
faut        0.000077
internet    0.000076
faire       0.000070
travail     0.000064
Name: tous, dtype: float32

education


...

faut        0.000938
exemple     0.000225
faire       0.000218
moyens      0.000188
non         0.000179
tout        0.000167
public      0.000140
publics     0.000138
service     0.000108
services    0.000078
Name: faut, dtype: float32

tout


tout         0.000956
monde        0.000465
temps        0.000233
non          0.000198
faire        0.000180
faut         0.000167
public       0.000112
service      0.000096
services     0.000070
nationale    0.000059
Name: tout, dtype: float32

police


police         0.000974
gendarmerie    0.000777
justice        0.000191
hopitaux       0.000093
nationale      0.000093
education      0.000073
sante          0.000062
services       0.000053
non                 NaN
caf                 NaN
Name: police, dtype: float32

In [70]:
gc.collect()

147

# Question 12 :  Connaissez-vous le "droit à l'erreur", c'est-à-dire le droit d'affirmer votre bonne foi lorsque vous faites un erreur dans vos déclarations ?

In [71]:
question_12 = 'Connaissez-vous le "droit à l\'erreur", c\'est-à-dire le droit d\'affirmer votre bonne foi lorsque vous faites un erreur dans vos déclarations ?'
qu_12,df_qu_12_one_hot,df_cov_12,df_confidence_12,df_lift_12 = compute_similarities(question_12, new_res)

In [72]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_12[i].sort_values(ascending=False).head(10))

oui


oui    23995
non        0
Name: oui, dtype: uint16

non


non    6164
oui       0
Name: non, dtype: uint16

In [73]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_12[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [74]:
for i in df_qu_12_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_12[i].sort_values(ascending=False).head(10))

oui


oui    0.000042
non         NaN
Name: oui, dtype: float32

non


non    0.000162
oui         NaN
Name: non, dtype: float32

In [75]:
gc.collect()

14

# Question 13 :  Si oui, avez-vous déjà utilisé ce droit à l'erreur ?

In [76]:
question_13 = "Si oui, avez-vous déjà utilisé ce droit à l'erreur ?"
qu_13,df_qu_13_one_hot,df_cov_13,df_confidence_13,df_lift_13 = compute_similarities(question_13, new_res)

In [77]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_13[i].sort_values(ascending=False).head(10))

non


non    24279
oui        0
Name: non, dtype: uint16

oui


oui    2217
non       0
Name: oui, dtype: uint16

In [78]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_13[i].sort_values(ascending=False).head(10))

non


non    1.0
oui    0.0
Name: non, dtype: float32

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

In [79]:
for i in df_qu_13_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_13[i].sort_values(ascending=False).head(10))

non


non    0.000041
oui         NaN
Name: non, dtype: float32

oui


oui    0.000451
non         NaN
Name: oui, dtype: float32

In [80]:
gc.collect()

14

# Question 14 :  Si oui, à quelle occasion en avez-vous fait usage ?

In [81]:
question_14 = "Si oui, à quelle occasion en avez-vous fait usage ?"
qu_14,df_qu_14_one_hot,df_cov_14,df_confidence_14,df_lift_14 = compute_similarities(question_14, new_res)

In [82]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_14[i].sort_values(ascending=False).head(10))

declaration


declaration    933
impots         308
erreur         290
impot          196
droit            0
Name: declaration, dtype: uint16

impots


impots         731
declaration    308
erreur         190
impot            0
droit            0
Name: impots, dtype: uint16

erreur


erreur         704
droit          293
declaration    290
impots         190
impot            0
Name: erreur, dtype: uint16

droit


droit          385
erreur         293
declaration      0
impot            0
impots           0
Name: droit, dtype: uint16

impot


impot          358
declaration    196
erreur           0
impots           0
droit            0
Name: impot, dtype: uint16

In [83]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_14[i].sort_values(ascending=False).head(10))

declaration


declaration    1.000000
impot          0.547486
impots         0.421341
erreur         0.411932
droit          0.000000
Name: declaration, dtype: float32

impots


impots         1.000000
declaration    0.330118
erreur         0.269886
impot          0.000000
droit          0.000000
Name: impots, dtype: float32

erreur


erreur         1.000000
droit          0.761039
declaration    0.310825
impots         0.259918
impot          0.000000
Name: erreur, dtype: float32

droit


droit          1.000000
erreur         0.416193
declaration    0.000000
impot          0.000000
impots         0.000000
Name: droit, dtype: float32

impot


impot          1.000000
declaration    0.210075
erreur         0.000000
impots         0.000000
droit          0.000000
Name: impot, dtype: float32

In [84]:
for i in df_qu_14_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_14[i].sort_values(ascending=False).head(10))

declaration


declaration    0.001072
impot          0.000587
impots         0.000452
erreur         0.000442
droit               NaN
Name: declaration, dtype: float32

impots


impots         0.001368
declaration    0.000452
erreur         0.000369
droit               NaN
impot               NaN
Name: impots, dtype: float32

erreur


erreur         0.001420
droit          0.001081
declaration    0.000442
impots         0.000369
impot               NaN
Name: erreur, dtype: float32

droit


droit          0.002597
erreur         0.001081
impots              NaN
impot               NaN
declaration         NaN
Name: droit, dtype: float32

impot


impot          0.002793
declaration    0.000587
droit               NaN
impots              NaN
erreur              NaN
Name: impot, dtype: float32

In [85]:
gc.collect()

196

# Question 15 :  Pouvez-vous identifier des règles que l'administration vous a déjà demandé d'appliquer et que vous avez jugées inutiles ou trop complexes ?

In [86]:
question_15 = "Pouvez-vous identifier des règles que l'administration vous a déjà demandé d'appliquer et que vous avez jugées inutiles ou trop complexes ?"
qu_15,df_qu_15_one_hot,df_cov_15,df_confidence_15,df_lift_15 = compute_similarities(question_15, new_res)

In [87]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_15[i].sort_values(ascending=False).head(10))

non


non             1734
faire            130
demande          112
caf                0
regles             0
droit              0
informations       0
tout               0
travail            0
entreprise         0
Name: non, dtype: uint16

demande


demande           1519
faire              241
dossier            205
administration     202
carte              197
documents          191
permis             178
tout               157
mois               152
travaux            140
Name: demande, dtype: uint16

faire


...

complexe       559
declaration    104
regles           0
identite         0
grise            0
emploi           0
pole             0
inutile          0
papier           0
80               0
Name: complexe, dtype: uint16

identite


identite          552
carte             352
renouvellement    137
passeport         127
demande           119
papier              0
inutile             0
regles              0
grise               0
emploi              0
Name: identite, dtype: uint16

travaux


travaux        540
declaration    187
demande        140
permis         128
construire     107
inutile          0
identite         0
grise            0
emploi           0
pole             0
Name: travaux, dtype: uint16

In [88]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_15[i].sort_values(ascending=False).head(10))

non


non             1.000000
faire           0.111492
demande         0.073733
caf             0.000000
regles          0.000000
droit           0.000000
informations    0.000000
tout            0.000000
travail         0.000000
entreprise      0.000000
Name: non, dtype: float32

demande


demande      1.000000
retraite     0.288591
mois         0.288425
travaux      0.259259
dossier      0.257538
2            0.254364
ans          0.251880
documents    0.246134
fournir      0.228814
identite     0.215580
Name: demande, dtype: float32

faire


...

complexe       1.00000
declaration    0.09747
regles         0.00000
identite       0.00000
grise          0.00000
emploi         0.00000
pole           0.00000
inutile        0.00000
papier         0.00000
80             0.00000
Name: complexe, dtype: float32

identite


identite          1.000000
passeport         0.427609
renouvellement    0.420245
carte             0.382193
demande           0.078341
papier            0.000000
inutile           0.000000
regles            0.000000
grise             0.000000
emploi            0.000000
Name: identite, dtype: float32

travaux


travaux        1.000000
declaration    0.175258
construire     0.158519
permis         0.128773
demande        0.092166
inutile        0.000000
identite       0.000000
grise          0.000000
emploi         0.000000
pole           0.000000
Name: travaux, dtype: float32

In [89]:
for i in df_qu_15_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_15[i].sort_values(ascending=False).head(10))

non


non         0.000577
faire       0.000064
demande     0.000043
fournir          NaN
caf              NaN
securite         NaN
obtenir          NaN
compte           NaN
carte            NaN
payer            NaN
Name: non, dtype: float32

demande


demande      0.000658
retraite     0.000190
mois         0.000190
travaux      0.000171
dossier      0.000170
2            0.000167
ans          0.000166
documents    0.000162
fournir      0.000151
identite     0.000142
Name: demande, dtype: float32

faire


...

complexe       0.001789
declaration    0.000174
non                 NaN
fournir             NaN
caf                 NaN
securite            NaN
obtenir             NaN
compte              NaN
carte               NaN
payer               NaN
Name: complexe, dtype: float32

identite


identite          0.001812
passeport         0.000775
renouvellement    0.000761
carte             0.000692
demande           0.000142
non                    NaN
fournir                NaN
caf                    NaN
securite               NaN
obtenir                NaN
Name: identite, dtype: float32

travaux


travaux        0.001852
declaration    0.000325
construire     0.000294
permis         0.000238
demande        0.000171
non                 NaN
fournir             NaN
caf                 NaN
securite            NaN
obtenir             NaN
Name: travaux, dtype: float32

In [90]:
gc.collect()

294

# Question 16 :  Faut-il donner plus d'autonomie aux fonctionnaires de terrain ?

In [91]:
question_16 = "Faut-il donner plus d'autonomie aux fonctionnaires de terrain ?"
qu_16,df_qu_16_one_hot,df_cov_16,df_confidence_16,df_lift_16 = compute_similarities(question_16, new_res)

In [92]:
for i in df_qu_16_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_16[i].sort_values(ascending=False).head(10))

oui


oui    18190
non        0
Name: oui, dtype: uint16

non


non    7489
oui       0
Name: non, dtype: uint16

In [93]:
for i in df_qu_16_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_16[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [94]:
for i in df_qu_16_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_16[i].sort_values(ascending=False).head(10))

oui


oui    0.000055
non         NaN
Name: oui, dtype: float32

non


non    0.000134
oui         NaN
Name: non, dtype: float32

In [95]:
gc.collect()

14

# Question 17 :   Si oui, comment ?

In [96]:
question_17 = "Si oui, comment ?_terrain"
qu_17,df_qu_17_one_hot,df_cov_17,df_confidence_17,df_lift_17 = compute_similarities(question_17, new_res)

In [97]:
for i in df_qu_17_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_17[i].sort_values(ascending=False).head(10))

fonctionnaires


fonctionnaires    1375
terrain            438
faut               205
faire              184
autonomie          170
non                126
donner             116
pouvoir            114
travail            112
tout               108
Name: fonctionnaires, dtype: uint16

terrain


terrain           1152
fonctionnaires     438
fonctionnaire      207
faut               145
autonomie          138
faire              128
pouvoir            101
public               0
cadre                0
regles               0
Name: terrain, dtype: uint16

pouvoir


...

responsabilisant    503
question              0
controle              0
decisions             0
regles                0
prendre               0
agents                0
cas                   0
responsabilite        0
competences           0
Name: responsabilisant, dtype: uint16

controle


controle          501
public              0
question            0
decisions           0
regles              0
prendre             0
agents              0
cas                 0
responsabilite      0
competences         0
Name: controle, dtype: uint16

services


services          493
public              0
tout                0
controle            0
decisions           0
regles              0
prendre             0
agents              0
cas                 0
responsabilite      0
Name: services, dtype: uint16

In [98]:
for i in df_qu_17_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_17[i].sort_values(ascending=False).head(10))

fonctionnaires


fonctionnaires    1.000000
terrain           0.380208
faut              0.225027
non               0.207578
travail           0.205882
tout              0.181208
faire             0.174905
autonomie         0.165049
donner            0.158038
pouvoir           0.106343
Name: fonctionnaires, dtype: float32

terrain


terrain           1.000000
fonctionnaires    0.318545
fonctionnaire     0.291549
faut              0.159166
autonomie         0.133981
faire             0.121673
pouvoir           0.094216
public            0.000000
cadre             0.000000
regles            0.000000
Name: terrain, dtype: float32

pouvoir


...

responsabilisant    1.0
question            0.0
controle            0.0
decisions           0.0
regles              0.0
prendre             0.0
agents              0.0
cas                 0.0
responsabilite      0.0
competences         0.0
Name: responsabilisant, dtype: float32

controle


controle          1.0
public            0.0
question          0.0
decisions         0.0
regles            0.0
prendre           0.0
agents            0.0
cas               0.0
responsabilite    0.0
competences       0.0
Name: controle, dtype: float32

services


services          1.0
public            0.0
tout              0.0
controle          0.0
decisions         0.0
regles            0.0
prendre           0.0
agents            0.0
cas               0.0
responsabilite    0.0
Name: services, dtype: float32

In [99]:
for i in df_qu_17_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_17[i].sort_values(ascending=False).head(10))

fonctionnaires


fonctionnaires    0.000727
terrain           0.000277
faut              0.000164
non               0.000151
travail           0.000150
tout              0.000132
faire             0.000127
autonomie         0.000120
donner            0.000115
pouvoir           0.000077
Name: fonctionnaires, dtype: float32

terrain


terrain           0.000868
fonctionnaires    0.000277
fonctionnaire     0.000253
faut              0.000138
autonomie         0.000116
faire             0.000106
pouvoir           0.000082
non                    NaN
decision               NaN
objectifs              NaN
Name: terrain, dtype: float32

pouvoir


...

responsabilisant    0.001988
non                      NaN
decision                 NaN
objectifs                NaN
permettre                NaN
sais                     NaN
compte                   NaN
fonction                 NaN
donnant                  NaN
pouvoir                  NaN
Name: responsabilisant, dtype: float32

controle


controle     0.001996
non               NaN
decision          NaN
objectifs         NaN
permettre         NaN
sais              NaN
compte            NaN
fonction          NaN
donnant           NaN
pouvoir           NaN
Name: controle, dtype: float32

services


services     0.002028
non               NaN
decision          NaN
objectifs         NaN
permettre         NaN
sais              NaN
compte            NaN
fonction          NaN
donnant           NaN
pouvoir           NaN
Name: services, dtype: float32

In [100]:
gc.collect()

285

# Question 18 :   Faut-il revoir le fonctionnement et la formation de l'administration ?

In [101]:
question_18 = "Faut-il revoir le fonctionnement et la formation de l'administration ?"
qu_18,df_qu_18_one_hot,df_cov_18,df_confidence_18,df_lift_18 = compute_similarities(question_18, new_res)

In [102]:
for i in df_qu_18_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_18[i].sort_values(ascending=False).head(10))

oui


oui    23090
non        0
Name: oui, dtype: uint16

non


non    2604
oui       0
Name: non, dtype: uint16

In [103]:
for i in df_qu_18_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_18[i].sort_values(ascending=False).head(10))

oui


oui    1.0
non    0.0
Name: oui, dtype: float32

non


non    1.0
oui    0.0
Name: non, dtype: float32

In [104]:
for i in df_qu_18_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_18[i].sort_values(ascending=False).head(10))

oui


oui    0.000043
non         NaN
Name: oui, dtype: float32

non


non    0.000384
oui         NaN
Name: non, dtype: float32

In [105]:
gc.collect()

14

# Question 19 :  Si oui, comment ?

In [106]:
question_19 = "Si oui, comment ?_admin"
qu_19,df_qu_19_one_hot,df_cov_19,df_confidence_19,df_lift_19 = compute_similarities(question_19, new_res)

In [107]:
for i in df_qu_19_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_19[i].sort_values(ascending=False).head(10))

fonctionnaires


fonctionnaires    3579
formation          590
administration     568
hauts              557
faut               530
prive              492
faire              472
service            410
tous               382
fonction           375
Name: fonctionnaires, dtype: uint16

formation


formation         2830
fonctionnaires     590
administration     408
agents             333
faut               322
continue           317
faire              285
service            277
fonctionnement     252
tout               250
Name: formation, dtype: uint16

administration


...

fonctionnaire     1139
statut             342
fonctionnaires     281
prive              234
administration     184
formation          177
service            174
vie                173
tout               164
fonction           163
Name: fonctionnaire, dtype: uint16

terrain


terrain           1098
fonctionnaires     306
formation          237
administration     137
faut               128
faire              127
agents             120
ena                117
cas                  0
travailler           0
Name: terrain, dtype: uint16

vie


vie               1011
emploi             379
fonctionnaires     350
prive              214
fonctionnaire      173
administration     163
formation          155
statut             148
supprimer          146
fonction           139
Name: vie, dtype: uint16

In [108]:
for i in df_qu_19_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_19[i].sort_values(ascending=False).head(10))

fonctionnaires


fonctionnaires    1.000000
hauts             0.948893
nombre            0.465659
devraient         0.415902
formes            0.406143
fonctions         0.393795
remuneration      0.387454
carriere          0.385749
secteur           0.362500
postes            0.361702
Name: fonctionnaires, dtype: float32

formation


formation         1.000000
continue          0.908309
meilleure         0.398649
carriere          0.316953
accueil           0.296392
besoins           0.281579
fonctionnement    0.274211
niveau            0.255613
revoir            0.251863
poste             0.239669
Name: formation, dtype: float32

administration


...

fonctionnaire    1.000000
statut           0.456000
emploi           0.178434
vie              0.171118
pouvoir          0.164134
publique         0.154696
concours         0.139198
fonction         0.138019
prive            0.137324
travail          0.128696
Name: fonctionnaire, dtype: float32

terrain


terrain           1.000000
ena               0.090208
fonctionnaires    0.085499
formation         0.083746
agents            0.080917
faire             0.073241
faut              0.072769
administration    0.059539
cas               0.000000
travailler        0.000000
Name: terrain, dtype: float32

vie


vie              1.000000
emploi           0.486521
fin              0.325513
statut           0.197333
fonctionnaire    0.151888
prive            0.125587
fonction         0.117697
supprimer        0.115598
publique         0.113812
tout             0.101657
Name: vie, dtype: float32

In [109]:
for i in df_qu_19_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_19[i].sort_values(ascending=False).head(10))

fonctionnaires


fonctionnaires    0.000279
hauts             0.000265
nombre            0.000130
devraient         0.000116
formes            0.000113
fonctions         0.000110
remuneration      0.000108
carriere          0.000108
secteur           0.000101
postes            0.000101
Name: fonctionnaires, dtype: float32

formation


formation         0.000353
continue          0.000321
meilleure         0.000141
carriere          0.000112
accueil           0.000105
besoins           0.000099
fonctionnement    0.000097
niveau            0.000090
revoir            0.000089
poste             0.000085
Name: formation, dtype: float32

administration


...

fonctionnaire    0.000878
statut           0.000400
emploi           0.000157
vie              0.000150
pouvoir          0.000144
publique         0.000136
concours         0.000122
fonction         0.000121
prive            0.000121
travail          0.000113
Name: fonctionnaire, dtype: float32

terrain


terrain           0.000911
ena               0.000082
fonctionnaires    0.000078
formation         0.000076
agents            0.000074
faire             0.000067
faut              0.000066
administration    0.000054
non                    NaN
eviter                 NaN
Name: terrain, dtype: float32

vie


vie              0.000989
emploi           0.000481
fin              0.000322
statut           0.000195
fonctionnaire    0.000150
prive            0.000124
fonction         0.000116
supprimer        0.000114
publique         0.000113
tout             0.000101
Name: vie, dtype: float32

In [110]:
gc.collect()

147

# Question 20:  Comment l'Etat et les collectivités territoriales peuvent-ils s'améliorer pour mieux répondre aux défis de nos territoires les plus en difficulté ?

In [111]:
question_20 = "Comment l'Etat et les collectivités territoriales peuvent-ils s'améliorer pour mieux répondre aux défis de nos territoires les plus en difficulté ?"
qu_20,df_qu_20_one_hot,df_cov_20,df_confidence_20,df_lift_20 = compute_similarities(question_20, new_res)

In [112]:
for i in df_qu_20_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_20[i].sort_values(ascending=False).head(10))

services


services       3093
publics        1334
territoires     518
proximite       398
maisons         367
tous            358
faut            346
faire           302
service         300
public          293
Name: services, dtype: uint16

territoires


territoires      2547
services          518
difficulte        418
faut              343
publics           309
difficultes       281
faire             267
collectivites     240
tous              233
ruraux            216
Name: territoires, dtype: uint16

faire


...

non              902
services         152
territoires      146
faire            142
faut             138
tout             115
collectivites    104
projets            0
elus               0
rurales            0
Name: non, dtype: uint16

fonctionnaires


fonctionnaires     889
services           155
faire              141
territoires        127
nombre             104
faut               102
politiques           0
administratives      0
petites              0
publics              0
Name: fonctionnaires, dtype: uint16

mieux


mieux            872
territoires      150
services         136
faire            131
collectivites    110
projets            0
elus               0
rurales            0
prendre            0
decisions          0
Name: mieux, dtype: uint16

In [113]:
for i in df_qu_20_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_20[i].sort_values(ascending=False).head(10))

services


services        1.000000
publics         0.856226
itinerants      0.755725
maisons         0.625213
publiques       0.541538
poste           0.532995
bus             0.469649
regroupement    0.457364
regrouper       0.438247
rurales         0.429907
Name: services, dtype: float32

territoires


territoires      1.000000
ruraux           0.710526
difficulte       0.601439
difficultes      0.418778
entreprises      0.285088
developpement    0.272727
grandes          0.251029
aider            0.243852
villes           0.238550
habitants        0.237991
Name: territoires, dtype: float32

faire


...

non              1.000000
faut             0.088803
tout             0.086207
faire            0.082847
collectivites    0.072172
territoires      0.057322
services         0.049143
developper       0.000000
proximite        0.000000
travail          0.000000
Name: non, dtype: float32

fonctionnaires


fonctionnaires     1.000000
nombre             0.169106
faire              0.082264
faut               0.065637
services           0.050113
territoires        0.049863
projets            0.000000
administratives    0.000000
petites            0.000000
publics            0.000000
Name: fonctionnaires, dtype: float32

mieux


mieux            1.000000
faire            0.076429
collectivites    0.076336
territoires      0.058893
services         0.043970
projets          0.000000
elus             0.000000
rurales          0.000000
prendre          0.000000
decisions        0.000000
Name: mieux, dtype: float32

In [114]:
for i in df_qu_20_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_20[i].sort_values(ascending=False).head(10))

services


services        0.000323
publics         0.000277
itinerants      0.000244
maisons         0.000202
publiques       0.000175
poste           0.000172
bus             0.000152
regroupement    0.000148
regrouper       0.000142
rurales         0.000139
Name: services, dtype: float32

territoires


territoires      0.000393
ruraux           0.000279
difficulte       0.000236
difficultes      0.000164
entreprises      0.000112
developpement    0.000107
grandes          0.000099
aider            0.000096
villes           0.000094
habitants        0.000093
Name: territoires, dtype: float32

faire


...

non              0.001109
faut             0.000098
tout             0.000096
faire            0.000092
collectivites    0.000080
territoires      0.000064
services         0.000054
decision              NaN
eviter                NaN
numerique             NaN
Name: non, dtype: float32

fonctionnaires


fonctionnaires    0.001125
nombre            0.000190
faire             0.000093
faut              0.000074
services          0.000056
territoires       0.000056
non                    NaN
decision               NaN
eviter                 NaN
numerique              NaN
Name: fonctionnaires, dtype: float32

mieux


mieux            0.001147
faire            0.000088
collectivites    0.000088
territoires      0.000068
services         0.000050
non                   NaN
decision              NaN
eviter                NaN
numerique             NaN
permettre             NaN
Name: mieux, dtype: float32

In [115]:
gc.collect()

147

# Question 21: Si vous avez été amené à chercher une formation, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [116]:
question_21 = "Si vous avez été amené à chercher une formation, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_21,df_qu_21_one_hot,df_cov_21,df_confidence_21,df_lift_21 = compute_similarities(question_21, new_res)

In [117]:
for i in df_qu_21_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_21[i].sort_values(ascending=False).head(10))

formation


formation          2558
emploi              977
pole                858
formations          486
faire               333
ans                 238
trouver             218
demande             214
tout                213
professionnelle     212
Name: formation, dtype: uint16

emploi


emploi        1770
pole          1549
formation      977
formations     446
faire          233
demande        168
aide           165
ans            159
tout           148
trouver        146
Name: emploi, dtype: uint16

pole


...

travail        286
formation      184
emploi         125
pole           104
demande          0
jamais           0
financement      0
tout             0
ans              0
faut             0
Name: travail, dtype: uint16

professionnelle


professionnelle    268
formation          212
demande              0
jamais               0
financement          0
emploi               0
pole                 0
travail              0
tout                 0
ans                  0
Name: professionnelle, dtype: uint16

jamais


jamais         257
formation      140
demande          0
financement      0
emploi           0
pole             0
travail          0
tout             0
ans              0
faut             0
Name: jamais, dtype: uint16

In [118]:
for i in df_qu_21_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_21[i].sort_values(ascending=False).head(10))

formation


formation          1.000000
professionnelle    0.791045
demande            0.694805
faire              0.650391
travail            0.643357
ans                0.639785
aide               0.598639
financement        0.583062
trouver            0.573684
emploi             0.551977
Name: formation, dtype: float32

emploi


emploi         1.000000
pole           0.979140
aide           0.561224
demande        0.545455
financement    0.475570
faire          0.455078
travail        0.437063
ans            0.427419
trouver        0.384211
formation      0.381939
Name: emploi, dtype: float32

pole


...

travail        1.000000
formation      0.071931
emploi         0.070621
pole           0.065740
demande        0.000000
jamais         0.000000
financement    0.000000
tout           0.000000
ans            0.000000
faut           0.000000
Name: travail, dtype: float32

professionnelle


professionnelle    1.000000
formation          0.082877
demande            0.000000
jamais             0.000000
financement        0.000000
emploi             0.000000
pole               0.000000
travail            0.000000
tout               0.000000
ans                0.000000
Name: professionnelle, dtype: float32

jamais


jamais         1.00000
formation      0.05473
demande        0.00000
financement    0.00000
emploi         0.00000
pole           0.00000
travail        0.00000
tout           0.00000
ans            0.00000
faut           0.00000
Name: jamais, dtype: float32

In [119]:
for i in df_qu_21_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_21[i].sort_values(ascending=False).head(10))

formation


formation          0.000391
professionnelle    0.000309
demande            0.000272
faire              0.000254
travail            0.000252
ans                0.000250
aide               0.000234
financement        0.000228
trouver            0.000224
emploi             0.000216
Name: formation, dtype: float32

emploi


emploi         0.000565
pole           0.000553
aide           0.000317
demande        0.000308
financement    0.000269
faire          0.000257
travail        0.000247
ans            0.000241
trouver        0.000217
formation      0.000216
Name: emploi, dtype: float32

pole


...

travail        0.003497
formation      0.000252
emploi         0.000247
pole           0.000230
non                 NaN
financement         NaN
tout                NaN
jamais              NaN
ans                 NaN
faire               NaN
Name: travail, dtype: float32

professionnelle


professionnelle    0.003731
formation          0.000309
non                     NaN
financement             NaN
emploi                  NaN
pole                    NaN
travail                 NaN
tout                    NaN
jamais                  NaN
ans                     NaN
Name: professionnelle, dtype: float32

jamais


jamais         0.003891
formation      0.000213
non                 NaN
financement         NaN
emploi              NaN
pole                NaN
travail             NaN
tout                NaN
ans                 NaN
faire               NaN
Name: jamais, dtype: float32

In [120]:
gc.collect()

126

# Question 22:  Si vous avez été amené à scolariser votre enfant, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [121]:
question_22 = "Si vous avez été amené à scolariser votre enfant, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_22,df_qu_22_one_hot,df_cov_22,df_confidence_22,df_lift_22 = compute_similarities(question_22, new_res)

In [122]:
for i in df_qu_22_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_22[i].sort_values(ascending=False).head(10))

enfants


enfants        1948
ecole           587
education       405
parents         321
nationale       307
enseignants     296
scolaire        289
prive           281
faire           273
enfant          242
Name: enfants, dtype: uint16

ecole


ecole          1677
enfants         587
enfant          347
publique        299
education       260
parents         243
enseignants     235
scolaire        234
primaire        232
faire           222
Name: ecole, dtype: uint16

education


...

difficulte     530
enfants        160
ecole          108
professeurs      0
difficultes      0
systeme          0
ans              0
probleme         0
etudes           0
classe           0
Name: difficulte, dtype: uint16

classe


classe         516
eleves         218
enfants        214
ecole          178
education      131
enfant         121
enseignants    113
classes        108
faire          105
tout             0
Name: classe, dtype: uint16

enseignement


enseignement    513
enfants         172
enseignants     129
education       117
ecole           114
prive           112
qualite         105
formation         0
travail           0
tout              0
Name: enseignement, dtype: uint16

In [123]:
for i in df_qu_22_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_22[i].sort_values(ascending=False).head(10))

enfants


enfants       1.000000
3             0.500000
2             0.493333
parents       0.480539
tous          0.460177
scolarite     0.452899
maternelle    0.452055
public        0.443459
prive         0.441824
etudes        0.423221
Name: enfants, dtype: float32

ecole


ecole         1.000000
publique      0.854286
maternelle    0.547945
primaire      0.516704
mairie        0.458084
3             0.409449
fils          0.392857
ans           0.372146
faut          0.365314
parents       0.363772
Name: ecole, dtype: float32

education


...

difficulte     1.000000
enfants        0.082136
ecole          0.064401
professeurs    0.000000
difficultes    0.000000
systeme        0.000000
ans            0.000000
probleme       0.000000
etudes         0.000000
classe         0.000000
Name: difficulte, dtype: float32

classe


classe         1.000000
eleves         0.277354
classes        0.223602
faire          0.157186
enseignants    0.131243
enfant         0.114151
enfants        0.109856
education      0.107114
ecole          0.106142
tout           0.000000
Name: classe, dtype: float32

enseignement


enseignement    1.000000
qualite         0.364583
prive           0.176101
enseignants     0.149826
education       0.095666
enfants         0.088296
ecole           0.067979
formation       0.000000
travail         0.000000
tout            0.000000
Name: enseignement, dtype: float32

In [124]:
for i in df_qu_22_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_22[i].sort_values(ascending=False).head(10))

enfants


enfants       0.000513
3             0.000257
2             0.000253
parents       0.000247
tous          0.000236
scolarite     0.000232
maternelle    0.000232
public        0.000228
prive         0.000227
etudes        0.000217
Name: enfants, dtype: float32

ecole


ecole         0.000596
publique      0.000509
maternelle    0.000327
primaire      0.000308
mairie        0.000273
3             0.000244
fils          0.000234
ans           0.000222
faut          0.000218
parents       0.000217
Name: ecole, dtype: float32

education


...

difficulte     0.001887
enfants        0.000155
ecole          0.000122
non                 NaN
carte               NaN
2                   NaN
enseignants         NaN
college             NaN
scolaire            NaN
classes             NaN
Name: difficulte, dtype: float32

classe


classe         0.001938
eleves         0.000538
classes        0.000433
faire          0.000305
enseignants    0.000254
enfant         0.000221
enfants        0.000213
education      0.000208
ecole          0.000206
non                 NaN
Name: classe, dtype: float32

enseignement


enseignement    0.001949
qualite         0.000711
prive           0.000343
enseignants     0.000292
education       0.000186
enfants         0.000172
ecole           0.000133
non                  NaN
carte                NaN
2                    NaN
Name: enseignement, dtype: float32

In [125]:
gc.collect()

285

# Question 23: Si vous avez été amené à chercher un emploi, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [126]:
question_23 = "Si vous avez été amené à chercher un emploi, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_23,df_qu_23_one_hot,df_cov_23,df_confidence_23,df_lift_23 = compute_similarities(question_23, new_res)

In [127]:
for i in df_qu_23_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_23[i].sort_values(ascending=False).head(10))

emploi


emploi       5314
pole         4601
recherche     750
travail       643
aide          580
formation     547
trouver       535
faire         521
jamais        487
chomage       431
Name: emploi, dtype: uint16

pole


pole         4684
emploi       4601
recherche     590
travail       586
aide          511
formation     484
faire         458
trouver       442
jamais        409
chomage       384
Name: pole, dtype: uint16

non


...

offres         441
emploi         378
pole           341
demande          0
aucun            0
entreprise       0
agents           0
trouve           0
suivi            0
competences      0
Name: offres, dtype: uint16

anpe


anpe           420
emploi         190
entreprises      0
entreprise       0
pole             0
agents           0
trouve           0
suivi            0
competences      0
aucun            0
Name: anpe, dtype: uint16

temps


temps          395
emploi         320
pole           278
demande          0
entreprises      0
entreprise       0
agents           0
trouve           0
suivi            0
competences      0
Name: temps, dtype: uint16

In [128]:
for i in df_qu_23_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_23[i].sort_values(ascending=False).head(10))

emploi


emploi         1.000000
pole           0.982280
demandeurs     0.959854
recherche      0.929368
site           0.914062
conseiller     0.889680
propose        0.881657
trouver        0.871336
conseillers    0.868966
aider          0.868421
Name: emploi, dtype: float32

pole


pole           1.000000
site           0.871094
emploi         0.865826
agents         0.839117
conseiller     0.829181
demandeurs     0.828467
conseillers    0.824138
propose        0.810651
aider          0.785714
chomeurs       0.782759
Name: pole, dtype: float32

non


...

offres         1.000000
pole           0.072801
emploi         0.071133
demande        0.000000
aucun          0.000000
entreprise     0.000000
agents         0.000000
trouve         0.000000
suivi          0.000000
competences    0.000000
Name: offres, dtype: float32

anpe


anpe           1.000000
emploi         0.035755
entreprises    0.000000
entreprise     0.000000
pole           0.000000
agents         0.000000
trouve         0.000000
suivi          0.000000
competences    0.000000
aucun          0.000000
Name: anpe, dtype: float32

temps


temps          1.000000
emploi         0.060218
pole           0.059351
demande        0.000000
entreprises    0.000000
entreprise     0.000000
agents         0.000000
trouve         0.000000
suivi          0.000000
competences    0.000000
Name: temps, dtype: float32

In [129]:
for i in df_qu_23_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_23[i].sort_values(ascending=False).head(10))

emploi


emploi         0.000188
pole           0.000185
demandeurs     0.000181
recherche      0.000175
site           0.000172
conseiller     0.000167
propose        0.000166
trouver        0.000164
conseillers    0.000164
aider          0.000163
Name: emploi, dtype: float32

pole


pole           0.000213
site           0.000186
emploi         0.000185
agents         0.000179
conseiller     0.000177
demandeurs     0.000177
conseillers    0.000176
propose        0.000173
aider          0.000168
chomeurs       0.000167
Name: pole, dtype: float32

non


...

offres        0.002268
pole          0.000165
emploi        0.000161
non                NaN
demandeurs         NaN
conseiller         NaN
chomage            NaN
poste              NaN
service            NaN
emplois            NaN
Name: offres, dtype: float32

anpe


anpe          0.002381
emploi        0.000085
non                NaN
demandeurs         NaN
conseiller         NaN
offres             NaN
chomage            NaN
poste              NaN
service            NaN
emplois            NaN
Name: anpe, dtype: float32

temps


temps         0.002532
emploi        0.000152
pole          0.000150
non                NaN
demandeurs         NaN
conseiller         NaN
offres             NaN
chomage            NaN
poste              NaN
service            NaN
Name: temps, dtype: float32

In [130]:
gc.collect()

294

# Question 24:  Si vous avez été amené à préparer votre retraite, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [131]:
question_24 = "Si vous avez été amené à préparer votre retraite, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_24,df_qu_24_one_hot,df_cov_24,df_confidence_24,df_lift_24 = compute_similarities(question_24, new_res)

In [132]:
for i in df_qu_24_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_24[i].sort_values(ascending=False).head(10))

retraite


retraite    3799
dossier      705
ans          579
mois         494
demande      423
tout         395
carsat       377
faire        376
carriere     351
caisses      332
Name: retraite, dtype: uint16

dossier


dossier     1335
retraite     705
mois         270
carsat       255
demande      205
faire        154
tout         153
internet     149
ans          149
cnav         148
Name: dossier, dtype: uint16

carsat


...

site            513
retraite        227
internet        157
demande           0
long              0
informations      0
tout              0
agirc             0
travail           0
6                 0
Name: site, dtype: uint16

services


services        495
retraite        227
mois              0
information       0
informations      0
tout              0
agirc             0
travail           0
6                 0
retraites         0
Name: services, dtype: uint16

service


service         493
retraite        211
demande           0
aucun             0
information       0
informations      0
tout              0
agirc             0
travail           0
6                 0
Name: service, dtype: uint16

In [133]:
for i in df_qu_24_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_24[i].sort_values(ascending=False).head(10))

retraite


retraite          1.000000
preparer          0.869048
complementaire    0.824427
depart            0.819121
montant           0.791667
caisse            0.757979
mois              0.681379
6                 0.675000
2                 0.649805
prendre           0.644269
Name: retraite, dtype: float32

dossier


dossier      1.000000
mois         0.372414
6            0.371429
demande      0.301471
documents    0.296569
temps        0.269880
carsat       0.251728
agirc        0.239080
faire        0.231928
cnav         0.226646
Name: dossier, dtype: float32

carsat


...

site            1.000000
internet        0.231222
retraite        0.059753
demande         0.000000
long            0.000000
informations    0.000000
tout            0.000000
agirc           0.000000
travail         0.000000
6               0.000000
Name: site, dtype: float32

services


services        1.000000
retraite        0.059753
mois            0.000000
information     0.000000
informations    0.000000
tout            0.000000
agirc           0.000000
travail         0.000000
6               0.000000
retraites       0.000000
Name: services, dtype: float32

service


service         1.000000
retraite        0.055541
demande         0.000000
aucun           0.000000
information     0.000000
informations    0.000000
tout            0.000000
agirc           0.000000
travail         0.000000
6               0.000000
Name: service, dtype: float32

In [134]:
for i in df_qu_24_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_24[i].sort_values(ascending=False).head(10))

retraite


retraite          0.000263
preparer          0.000229
complementaire    0.000217
depart            0.000216
montant           0.000208
caisse            0.000200
mois              0.000179
6                 0.000178
2                 0.000171
prendre           0.000170
Name: retraite, dtype: float32

dossier


dossier      0.000749
mois         0.000279
6            0.000278
demande      0.000226
documents    0.000222
temps        0.000202
carsat       0.000189
agirc        0.000179
faire        0.000174
cnav         0.000170
Name: dossier, dtype: float32

carsat


...

site              0.001949
internet          0.000451
retraite          0.000116
non                    NaN
carsat                 NaN
complementaire         NaN
obtenir                NaN
compte                 NaN
arrco                  NaN
annees                 NaN
Name: site, dtype: float32

services


services          0.002020
retraite          0.000121
non                    NaN
carsat                 NaN
complementaire         NaN
obtenir                NaN
compte                 NaN
arrco                  NaN
annees                 NaN
2                      NaN
Name: services, dtype: float32

service


service           0.002028
retraite          0.000113
non                    NaN
carsat                 NaN
complementaire         NaN
obtenir                NaN
compte                 NaN
arrco                  NaN
annees                 NaN
2                      NaN
Name: service, dtype: float32

In [135]:
gc.collect()

294

# Question 25:  Si vous avez été amené à demander un remboursement de soins de santé, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [136]:
question_25 = "Si vous avez été amené à demander un remboursement de soins de santé, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_25,df_qu_25_one_hot,df_cov_25,df_confidence_25,df_lift_25 = compute_similarities(question_25, new_res)

In [137]:
for i in df_qu_25_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_25[i].sort_values(ascending=False).head(10))

remboursement


remboursement    1647
soins             398
securite          293
sociale           289
mutuelle          265
sante             250
cpam              204
carte             164
mutuelles         157
charge            149
Name: remboursement, dtype: uint16

securite


securite          1494
sociale           1432
mutuelle           324
remboursement      293
soins              219
remboursements     208
mutuelles          181
sante              171
charge             158
probleme           138
Name: securite, dtype: uint16

sociale


...

aucun        611
probleme     351
demande        0
long           0
secu           0
efficace       0
tiers          0
rembourse      0
payant         0
charge         0
Name: aucun, dtype: uint16

secu


secu             602
mutuelle         154
remboursement    109
demande            0
long               0
tout               0
efficace           0
tiers              0
rembourse          0
payant             0
Name: secu, dtype: uint16

faut


faut         581
soins        115
sociale      115
faire        107
sante        107
securite     107
mutuelle     106
rembourse      0
payant         0
long           0
Name: faut, dtype: uint16

In [138]:
for i in df_qu_25_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_25[i].sort_values(ascending=False).head(10))

remboursement


remboursement    1.000000
rapide           0.437500
delais           0.402256
lunettes         0.360714
dentaires        0.328125
frais            0.327982
soins            0.316627
demande          0.312834
medecin          0.255000
systeme          0.245681
Name: remboursement, dtype: float32

securite


securite          1.000000
sociale           0.961074
mutuelle          0.243243
mutuelles         0.219394
charge            0.198742
mois              0.195298
faire             0.192201
remboursements    0.185218
faut              0.184165
remboursement     0.177899
Name: securite, dtype: float32

sociale


...

aucun        1.000000
probleme     0.306016
demande      0.000000
long         0.000000
secu         0.000000
efficace     0.000000
tiers        0.000000
rembourse    0.000000
payant       0.000000
charge       0.000000
Name: aucun, dtype: float32

secu


secu             1.000000
mutuelle         0.115616
remboursement    0.066181
demande          0.000000
long             0.000000
tout             0.000000
efficace         0.000000
tiers            0.000000
rembourse        0.000000
payant           0.000000
Name: secu, dtype: float32

faut


faut         1.000000
faire        0.149025
sante        0.097008
soins        0.091488
mutuelle     0.079580
sociale      0.077181
securite     0.071620
efficace     0.000000
tiers        0.000000
rembourse    0.000000
Name: faut, dtype: float32

In [139]:
for i in df_qu_25_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_25[i].sort_values(ascending=False).head(10))

remboursement


remboursement    0.000607
rapide           0.000266
delais           0.000244
lunettes         0.000219
dentaires        0.000199
frais            0.000199
soins            0.000192
demande          0.000190
medecin          0.000155
systeme          0.000149
Name: remboursement, dtype: float32

securite


securite          0.000669
sociale           0.000643
mutuelle          0.000163
mutuelles         0.000147
charge            0.000133
mois              0.000131
faire             0.000129
remboursements    0.000124
faut              0.000123
remboursement     0.000119
Name: securite, dtype: float32

sociale


...

aucun             0.001637
probleme          0.000501
non                    NaN
securite               NaN
fonctionne             NaN
remboursements         NaN
compte                 NaN
carte                  NaN
payer                  NaN
rembourser             NaN
Name: aucun, dtype: float32

secu


secu              0.001661
mutuelle          0.000192
remboursement     0.000110
non                    NaN
securite               NaN
fonctionne             NaN
remboursements         NaN
compte                 NaN
carte                  NaN
payer                  NaN
Name: secu, dtype: float32

faut


faut              0.001721
faire             0.000256
sante             0.000167
soins             0.000157
mutuelle          0.000137
sociale           0.000133
securite          0.000123
non                    NaN
fonctionne             NaN
remboursements         NaN
Name: faut, dtype: float32

In [140]:
gc.collect()

285

# Question 26:  Si vous avez été amené à faire une demande d'aide pour une situation de handicap, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [141]:
question_26 = "Si vous avez été amené à faire une demande d'aide pour une situation de handicap, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_26,df_qu_26_one_hot,df_cov_26,df_confidence_26,df_lift_26 = compute_similarities(question_26, new_res)

In [142]:
for i in df_qu_26_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_26[i].sort_values(ascending=False).head(10))

non


non          1008
concerne      392
mois            0
reponse         0
long            0
handicap        0
handicape       0
tout            0
dossiers        0
demande         0
Name: non, dtype: uint16

mdph


mdph        980
dossier     285
handicap    213
demande     182
mois        174
dossiers    157
ans         130
faire       129
aide        123
long        121
Name: mdph, dtype: uint16

concerne


...

tous         259
ans          148
handicap     105
mdph         103
demande        0
dossiers       0
reponse        0
long           0
handicape      0
tout           0
Name: tous, dtype: uint16

faut


faut         258
demande        0
reponse        0
long           0
handicap       0
handicape      0
tout           0
dossiers       0
mois           0
dossier        0
Name: faut, dtype: uint16

ras


ras          258
demande        0
mois           0
reponse        0
long           0
handicap       0
handicape      0
tout           0
dossiers       0
dossier        0
Name: ras, dtype: uint16

In [143]:
for i in df_qu_26_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_26[i].sort_values(ascending=False).head(10))

non


non          1.000000
concerne     0.507115
mois         0.000000
reponse      0.000000
long         0.000000
handicap     0.000000
handicape    0.000000
tout         0.000000
dossiers     0.000000
demande      0.000000
Name: non, dtype: float32

mdph


mdph        1.000000
dossiers    0.539519
dossier     0.475000
mois        0.429630
long        0.408784
tous        0.397683
faire       0.357341
ans         0.350404
demande     0.338290
handicap    0.306475
Name: mdph, dtype: float32

concerne


...

tous         1.000000
ans          0.398922
handicap     0.151079
mdph         0.105102
demande      0.000000
dossiers     0.000000
reponse      0.000000
long         0.000000
handicape    0.000000
tout         0.000000
Name: tous, dtype: float32

faut


faut         1.0
demande      0.0
reponse      0.0
long         0.0
handicap     0.0
handicape    0.0
tout         0.0
dossiers     0.0
mois         0.0
dossier      0.0
Name: faut, dtype: float32

ras


ras          1.0
demande      0.0
mois         0.0
reponse      0.0
long         0.0
handicap     0.0
handicape    0.0
tout         0.0
dossiers     0.0
dossier      0.0
Name: ras, dtype: float32

In [144]:
for i in df_qu_26_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_26[i].sort_values(ascending=False).head(10))

non


non          0.000992
concerne     0.000503
reponse           NaN
long              NaN
handicap          NaN
handicape         NaN
tout              NaN
dossiers          NaN
mois              NaN
dossier           NaN
Name: non, dtype: float32

mdph


mdph        0.001020
dossiers    0.000551
dossier     0.000485
mois        0.000438
long        0.000417
tous        0.000406
faire       0.000365
ans         0.000358
demande     0.000345
handicap    0.000313
Name: mdph, dtype: float32

concerne


...

tous         0.003861
ans          0.001540
handicap     0.000583
mdph         0.000406
non               NaN
reponse           NaN
long              NaN
handicape         NaN
tout              NaN
dossiers          NaN
Name: tous, dtype: float32

faut


faut         0.003876
non               NaN
reponse           NaN
long              NaN
handicap          NaN
handicape         NaN
tout              NaN
dossiers          NaN
mois              NaN
dossier           NaN
Name: faut, dtype: float32

ras


ras          0.003876
non               NaN
reponse           NaN
long              NaN
handicap          NaN
handicape         NaN
tout              NaN
dossiers          NaN
mois              NaN
dossier           NaN
Name: ras, dtype: float32

In [145]:
gc.collect()

133

# Question 27: Si vous avez été amené à créer une entreprise, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [146]:
question_27 = "Si vous avez été amené à créer une entreprise, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_27,df_qu_27_one_hot,df_cov_27,df_confidence_27,df_lift_27 = compute_similarities(question_27, new_res)

In [147]:
for i in df_qu_27_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_27[i].sort_values(ascending=False).head(10))

entreprise


entreprise    1076
creation       401
creer          285
auto           147
urssaf         144
faire          133
tout           120
aide           102
activite         0
impots           0
Name: entreprise, dtype: uint16

non


non            739
concerne       279
entreprise       0
impots           0
creation         0
demarches        0
entreprises      0
activite         0
rsi              0
faut             0
Name: non, dtype: uint16

creation


...

entreprises    278
faut             0
aide             0
impots           0
creation         0
demarches        0
activite         0
rsi              0
entreprise       0
tout             0
Name: entreprises, dtype: uint16

aide


aide           273
entreprise     102
faut             0
impots           0
creation         0
demarches        0
entreprises      0
activite         0
rsi              0
tout             0
Name: aide, dtype: uint16

activite


activite       255
faut             0
aide             0
impots           0
creation         0
demarches        0
entreprises      0
rsi              0
entreprise       0
tout             0
Name: activite, dtype: uint16

In [148]:
for i in df_qu_27_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_27[i].sort_values(ascending=False).head(10))

entreprise


entreprise    1.000000
creer         0.688406
creation      0.581159
auto          0.394102
aide          0.373626
faire         0.368421
tout          0.319149
urssaf        0.264220
activite      0.000000
impots        0.000000
Name: entreprise, dtype: float32

non


non            1.000000
concerne       0.555777
entreprise     0.000000
impots         0.000000
creation       0.000000
demarches      0.000000
entreprises    0.000000
activite       0.000000
rsi            0.000000
faut           0.000000
Name: non, dtype: float32

creation


...

entreprises    1.0
faut           0.0
aide           0.0
impots         0.0
creation       0.0
demarches      0.0
activite       0.0
rsi            0.0
entreprise     0.0
tout           0.0
Name: entreprises, dtype: float32

aide


aide           1.000000
entreprise     0.094796
faut           0.000000
impots         0.000000
creation       0.000000
demarches      0.000000
entreprises    0.000000
activite       0.000000
rsi            0.000000
tout           0.000000
Name: aide, dtype: float32

activite


activite       1.0
faut           0.0
aide           0.0
impots         0.0
creation       0.0
demarches      0.0
entreprises    0.0
rsi            0.0
entreprise     0.0
tout           0.0
Name: activite, dtype: float32

In [149]:
for i in df_qu_27_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_27[i].sort_values(ascending=False).head(10))

entreprise


entreprise    0.000929
creer         0.000640
creation      0.000540
auto          0.000366
aide          0.000347
faire         0.000342
tout          0.000297
urssaf        0.000246
non                NaN
impots             NaN
Name: entreprise, dtype: float32

non


non            0.001353
concerne       0.000752
impots              NaN
creation            NaN
demarches           NaN
entreprises         NaN
activite            NaN
rsi                 NaN
entreprise          NaN
tout                NaN
Name: non, dtype: float32

creation


...

entreprises     0.003597
non                  NaN
impots               NaN
creation             NaN
demarches            NaN
activite             NaN
rsi                  NaN
entreprise           NaN
tout                 NaN
entrepreneur         NaN
Name: entreprises, dtype: float32

aide


aide           0.003663
entreprise     0.000347
non                 NaN
impots              NaN
creation            NaN
demarches           NaN
entreprises         NaN
activite            NaN
rsi                 NaN
tout                NaN
Name: aide, dtype: float32

activite


activite        0.003922
non                  NaN
impots               NaN
creation             NaN
demarches            NaN
entreprises          NaN
rsi                  NaN
entreprise           NaN
tout                 NaN
entrepreneur         NaN
Name: activite, dtype: float32

In [150]:
gc.collect()

133

# Question 28: Si vous avez été amené à recruter du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [151]:
question_28 = "Si vous avez été amené à recruter du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_28,df_qu_28_one_hot,df_cov_28,df_confidence_28,df_lift_28 = compute_similarities(question_28, new_res)

In [152]:
for i in df_qu_28_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_28[i].sort_values(ascending=False).head(10))

emploi


emploi       1178
pole          979
travail       117
recruter      111
concerne        0
personnel       0
non             0
Name: emploi, dtype: uint16

pole


pole         997
emploi       979
concerne       0
recruter       0
personnel      0
travail        0
non            0
Name: pole, dtype: uint16

non


...

travail      379
emploi       117
concerne       0
recruter       0
personnel      0
pole           0
non            0
Name: travail, dtype: uint16

personnel


personnel    284
concerne       0
recruter       0
travail        0
pole           0
emploi         0
non            0
Name: personnel, dtype: uint16

recruter


recruter     255
emploi       111
concerne       0
personnel      0
travail        0
pole           0
non            0
Name: recruter, dtype: uint16

In [153]:
for i in df_qu_28_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_28[i].sort_values(ascending=False).head(10))

emploi


emploi       1.000000
pole         0.981946
recruter     0.435294
travail      0.308707
concerne     0.000000
personnel    0.000000
non          0.000000
Name: emploi, dtype: float32

pole


pole         1.00000
emploi       0.83107
concerne     0.00000
recruter     0.00000
personnel    0.00000
travail      0.00000
non          0.00000
Name: pole, dtype: float32

non


...

travail      1.000000
emploi       0.099321
concerne     0.000000
recruter     0.000000
personnel    0.000000
pole         0.000000
non          0.000000
Name: travail, dtype: float32

personnel


personnel    1.0
concerne     0.0
recruter     0.0
travail      0.0
pole         0.0
emploi       0.0
non          0.0
Name: personnel, dtype: float32

recruter


recruter     1.000000
emploi       0.094228
concerne     0.000000
personnel    0.000000
travail      0.000000
pole         0.000000
non          0.000000
Name: recruter, dtype: float32

In [154]:
for i in df_qu_28_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_28[i].sort_values(ascending=False).head(10))

emploi


emploi       0.000849
pole         0.000834
recruter     0.000370
travail      0.000262
non               NaN
personnel         NaN
concerne          NaN
Name: emploi, dtype: float32

pole


pole         0.001003
emploi       0.000834
non               NaN
travail           NaN
personnel         NaN
recruter          NaN
concerne          NaN
Name: pole, dtype: float32

non


...

travail      0.002639
emploi       0.000262
non               NaN
pole              NaN
personnel         NaN
recruter          NaN
concerne          NaN
Name: travail, dtype: float32

personnel


personnel    0.003521
non               NaN
emploi            NaN
pole              NaN
travail           NaN
recruter          NaN
concerne          NaN
Name: personnel, dtype: float32

recruter


recruter     0.003922
emploi       0.000370
non               NaN
pole              NaN
travail           NaN
personnel         NaN
concerne          NaN
Name: recruter, dtype: float32

In [155]:
gc.collect()

238

# Question 29: Si vous avez été amené à former du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [156]:
question_29 = "Si vous avez été amené à former du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_29,df_qu_29_one_hot,df_cov_29,df_confidence_29,df_lift_29 = compute_similarities(question_29, new_res)

In [157]:
for i in df_qu_29_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_29[i].sort_values(ascending=False).head(10))

formation


formation     718
formations    115
concerne        0
non             0
Name: formation, dtype: uint16

non


non           634
concerne      259
formations      0
formation       0
Name: non, dtype: uint16

concerne


concerne      474
non           259
formations      0
formation       0
Name: concerne, dtype: uint16

formations


formations    318
formation     115
concerne        0
non             0
Name: formations, dtype: uint16

In [158]:
for i in df_qu_29_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_29[i].sort_values(ascending=False).head(10))

formation


formation     1.000000
formations    0.361635
concerne      0.000000
non           0.000000
Name: formation, dtype: float32

non


non           1.000000
concerne      0.546413
formations    0.000000
formation     0.000000
Name: non, dtype: float32

concerne


concerne      1.000000
non           0.408517
formations    0.000000
formation     0.000000
Name: concerne, dtype: float32

formations


formations    1.000000
formation     0.160167
concerne      0.000000
non           0.000000
Name: formations, dtype: float32

In [159]:
for i in df_qu_29_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_29[i].sort_values(ascending=False).head(10))

formation


formation     0.001393
formations    0.000504
non                NaN
concerne           NaN
Name: formation, dtype: float32

non


non           0.001577
concerne      0.000862
formation          NaN
formations         NaN
Name: non, dtype: float32

concerne


concerne      0.002110
non           0.000862
formation          NaN
formations         NaN
Name: concerne, dtype: float32

formations


formations    0.003145
formation     0.000504
non                NaN
concerne           NaN
Name: formations, dtype: float32

In [160]:
gc.collect()

175

# Question 30:  Si vous avez été amené à rémunérer du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [161]:
question_30 = "Si vous avez été amené à rémunérer du personnel, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_30,df_qu_30_one_hot,df_cov_30,df_confidence_30,df_lift_30 = compute_similarities(question_30, new_res)

In [162]:
for i in df_qu_30_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_30[i].sort_values(ascending=False).head(10))

non


non         603
concerne    252
urssaf        0
cesu          0
travail       0
charges       0
Name: non, dtype: uint16

charges


charges     444
concerne      0
urssaf        0
cesu          0
travail       0
non           0
Name: charges, dtype: uint16

concerne


...

urssaf      290
concerne      0
cesu          0
travail       0
charges       0
non           0
Name: urssaf, dtype: uint16

cesu


cesu        284
concerne      0
urssaf        0
travail       0
charges       0
non           0
Name: cesu, dtype: uint16

travail


travail     255
concerne      0
urssaf        0
cesu          0
charges       0
non           0
Name: travail, dtype: uint16

In [163]:
for i in df_qu_30_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_30[i].sort_values(ascending=False).head(10))

non


non         1.000000
concerne    0.570136
urssaf      0.000000
cesu        0.000000
travail     0.000000
charges     0.000000
Name: non, dtype: float32

charges


charges     1.0
concerne    0.0
urssaf      0.0
cesu        0.0
travail     0.0
non         0.0
Name: charges, dtype: float32

concerne


...

urssaf      1.0
concerne    0.0
cesu        0.0
travail     0.0
charges     0.0
non         0.0
Name: urssaf, dtype: float32

cesu


cesu        1.0
concerne    0.0
urssaf      0.0
travail     0.0
charges     0.0
non         0.0
Name: cesu, dtype: float32

travail


travail     1.0
concerne    0.0
urssaf      0.0
cesu        0.0
charges     0.0
non         0.0
Name: travail, dtype: float32

In [164]:
for i in df_qu_30_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_30[i].sort_values(ascending=False).head(10))

non


non         0.001658
concerne    0.000945
charges          NaN
travail          NaN
cesu             NaN
urssaf           NaN
Name: non, dtype: float32

charges


charges     0.002252
non              NaN
travail          NaN
cesu             NaN
urssaf           NaN
concerne         NaN
Name: charges, dtype: float32

concerne


...

urssaf      0.003448
non              NaN
charges          NaN
travail          NaN
cesu             NaN
concerne         NaN
Name: urssaf, dtype: float32

cesu


cesu        0.003521
non              NaN
charges          NaN
travail          NaN
urssaf           NaN
concerne         NaN
Name: cesu, dtype: float32

travail


travail     0.003922
non              NaN
charges          NaN
cesu             NaN
urssaf           NaN
concerne         NaN
Name: travail, dtype: float32

In [165]:
gc.collect()

217

# Question 31: Si vous avez été amené à mettre fin à votre activité, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [166]:
question_31 = "Si vous avez été amené à mettre fin à votre activité, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_31,df_qu_31_one_hot,df_cov_31,df_confidence_31,df_lift_31 = compute_similarities(question_31, new_res)

In [167]:
for i in df_qu_31_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_31[i].sort_values(ascending=False).head(10))

non


non         627
concerne    251
activite      0
Name: non, dtype: uint16

concerne


concerne    441
non         251
activite      0
Name: concerne, dtype: uint16

activite


activite    330
concerne      0
non           0
Name: activite, dtype: uint16

In [168]:
for i in df_qu_31_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_31[i].sort_values(ascending=False).head(10))

non


non         1.000000
concerne    0.569161
activite    0.000000
Name: non, dtype: float32

concerne


concerne    1.000000
non         0.400319
activite    0.000000
Name: concerne, dtype: float32

activite


activite    1.0
concerne    0.0
non         0.0
Name: activite, dtype: float32

In [169]:
for i in df_qu_31_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_31[i].sort_values(ascending=False).head(10))

non


non         0.001595
concerne    0.000908
activite         NaN
Name: non, dtype: float32

concerne


concerne    0.002268
non         0.000908
activite         NaN
Name: concerne, dtype: float32

activite


activite    0.00303
non             NaN
concerne        NaN
Name: activite, dtype: float32

In [170]:
gc.collect()

21

# Question 32:  Si vous avez été amené à recruter une personne portant un handicap, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :

In [171]:
question_32 = "Si vous avez été amené à recruter une personne portant un handicap, pouvez-vous indiquer les éléments de satisfaction et/ou les difficultés rencontrés en précisant, pour chaque point, l'administration concernée :"
qu_32,df_qu_32_one_hot,df_cov_32,df_confidence_32,df_lift_32 = compute_similarities(question_32, new_res)

In [172]:
for i in df_qu_32_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_32[i].sort_values(ascending=False).head(10))

non


non         698
concerne    304
Name: non, dtype: uint16

concerne


concerne    528
non         304
Name: concerne, dtype: uint16

In [173]:
for i in df_qu_32_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_32[i].sort_values(ascending=False).head(10))

non


non         1.000000
concerne    0.575758
Name: non, dtype: float32

concerne


concerne    1.00000
non         0.43553
Name: concerne, dtype: float32

In [174]:
for i in df_qu_32_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_32[i].sort_values(ascending=False).head(10))

non


non         0.001433
concerne    0.000825
Name: non, dtype: float32

concerne


concerne    0.001894
non         0.000825
Name: concerne, dtype: float32

In [175]:
gc.collect()

14

# Question 33:  Y a-t-il d'autres points sur l'organisation de l'Etat et des services publics sur lesquels vous souhaiteriez vous exprimer ?

In [176]:
question_33 = "Y a-t-il d'autres points sur l'organisation de l'Etat et des services publics sur lesquels vous souhaiteriez vous exprimer ?"
qu_33,df_qu_33_one_hot,df_cov_33,df_confidence_33,df_lift_33 = compute_similarities(question_33, new_res)

In [177]:
for i in df_qu_33_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_cov_33[i].sort_values(ascending=False).head(10))

faut


faut        7970
faire       6050
tout        5961
service     5654
temps       5650
francais    5620
citoyens    5604
mettre      5571
securite    5534
vitesse     5527
Name: faut, dtype: uint16

faire


faire        7825
faut         6050
tout         6014
temps        5715
service      5705
citoyens     5631
francais     5630
mettre       5606
securite     5594
politique    5511
Name: faire, dtype: uint16

tout


...

mesure        5740
vitesse       5485
faut          5477
80            5458
km            5454
limitation    5441
faire         5408
francais      5400
reseau        5394
securite      5386
Name: mesure, dtype: uint16

revenir


revenir        5721
faut           5496
vitesse        5449
km             5432
limitation     5428
80             5423
mesure         5385
abaissement    5382
reseau         5379
secondaire     5371
Name: revenir, dtype: uint16

route


route          5641
vitesse        5496
80             5436
km             5426
securite       5408
limitation     5393
routes         5392
faire          5388
routiere       5374
conducteurs    5366
Name: route, dtype: uint16

In [178]:
for i in df_qu_33_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_confidence_33[i].sort_values(ascending=False).head(10))

faut


faut               1.000000
biaisee            0.994160
flashent           0.993873
probants           0.993602
rallonges          0.992615
injustement        0.992453
colles             0.991709
detourner          0.990714
experimentation    0.990099
imposee            0.989803
Name: faut, dtype: float32

faire


faire          1.000000
racket         0.992060
flashent       0.990427
detourner      0.989956
obligeant      0.988447
inutilement    0.988264
rives          0.988260
reduite        0.986578
rallonges      0.986366
injustement    0.986226
Name: faire, dtype: float32

tout


...

mesure             1.000000
biaisee            0.998493
probants           0.998306
flashent           0.996171
injustement        0.995472
rallonges          0.995266
colles             0.995101
experimentation    0.994769
imposee            0.993326
detourner          0.991851
Name: mesure, dtype: float32

revenir


revenir            1.000000
flashent           0.994831
biaisee            0.993783
probants           0.993414
rallonges          0.992804
injustement        0.992264
abaissement        0.991891
colles             0.991521
detourner          0.989388
experimentation    0.988791
Name: revenir, dtype: float32

route


route          1.000000
detourner      0.998294
rives          0.998106
obligeant      0.995833
inutilement    0.994700
reduite        0.993384
compteur       0.992114
laissez        0.992075
flashent       0.990618
colles         0.987564
Name: route, dtype: float32

In [179]:
for i in df_qu_33_one_hot.sum(axis=0).sort_values(ascending=False).head(20).index:
    print(i)
    display(df_lift_33[i].sort_values(ascending=False).head(10))

faut


faut               0.000125
biaisee            0.000125
flashent           0.000125
probants           0.000125
rallonges          0.000125
injustement        0.000125
colles             0.000124
detourner          0.000124
experimentation    0.000124
imposee            0.000124
Name: faut, dtype: float32

faire


faire          0.000128
racket         0.000127
flashent       0.000127
detourner      0.000127
obligeant      0.000126
inutilement    0.000126
rives          0.000126
reduite        0.000126
rallonges      0.000126
injustement    0.000126
Name: faire, dtype: float32

tout


...

mesure             0.000174
biaisee            0.000174
probants           0.000174
flashent           0.000174
injustement        0.000173
rallonges          0.000173
colles             0.000173
experimentation    0.000173
imposee            0.000173
detourner          0.000173
Name: mesure, dtype: float32

revenir


revenir            0.000175
flashent           0.000174
biaisee            0.000174
probants           0.000174
rallonges          0.000174
injustement        0.000173
abaissement        0.000173
colles             0.000173
detourner          0.000173
experimentation    0.000173
Name: revenir, dtype: float32

route


route          0.000177
detourner      0.000177
rives          0.000177
obligeant      0.000177
inutilement    0.000176
reduite        0.000176
compteur       0.000176
laissez        0.000176
flashent       0.000176
colles         0.000175
Name: route, dtype: float32

In [180]:
gc.collect()

105